# Guide to Analyze Spike Properties
This file is meant to ilustrate the data collected in each single spike using SpykProps.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/M.Sc/THESIS/ImageAnalysis/SpikeProperties/Spyk_Prop')
sys.path

In [ ]:
pip install skan

# Dependencies

In [ ]:
from SpykFunctions import *

In [ ]:
# Import dependencies


from SpykFunctions import *

from glob import glob
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import math
import pathlib

from skimage import measure, segmentation, color
from skimage.morphology import skeletonize, thin
from skimage.future import graph
from skimage.segmentation import watershed, active_contour
from skimage.feature import peak_local_max
from skimage.filters import meijering, sato, frangi, hessian, gaussian
from skimage.color import rgb2gray
from skimage.transform import hough_line, hough_line_peaks, probabilistic_hough_line
from skimage.draw import line

from PIL import Image

from skan import Skeleton, summarize, skeleton_to_csgraph, draw

import seaborn as sns

import imutils

import imageio

import random



# import astropy.units as u
# from fil_finder import FilFinder2D
# from astropy.io import fits

# Make sure Jupyter Notebook shows all outputs from the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Change the "%matplotlib inline" figure resolution on the notebook
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 300

# Get files

In [ ]:
# Define image folder
mypath = r'./Images/TEST'
# mypath = r'/content/drive/MyDrive/M.Sc/THESIS/ImageAnalysis/SpikeProperties/Spyk_Prop/Images/TEST'
Images = glob.glob(mypath + '/**/*.tif', recursive=True)

In [ ]:
len(Images)

## Select an image

In [ ]:
%%time
# Open and image and remove background
img0_name = Images[3]
# plt.imshow(plt.imread(img0_name))
img0 = RemoveBackground(img0_name, OtsuScaling=0.25)

In [ ]:
# plt.imshow(plt.imread(img0_name))
plt.imshow(img0)

In [ ]:
# mpl.rcParams['figure.dpi']= 300
# plt.imshow(img0)

In [ ]:
# Convert to gray
gray0 = img0 @ [0.2126, 0.7152, 0.0722]
   
# Threshold
otsu = filters.threshold_otsu(gray0)
bw0 = gray0 > 0
bw1 = morphology.remove_small_objects(bw0, min_size=1.5e-05 * gray0.shape[0] * gray0.shape[1])

# Get Lab values
img1 = np.where(bw1[..., None], img0, 0)
Lab = color.rgb2lab(img1)

In [ ]:
# plt.imshow(Lab)

In [ ]:
plt.imshow(bw1)
otsu

In [ ]:
# plt.imshow(bw1)

## Enumerate (label) spikes

**NOTE:** Spike number starts at 0 in TROE2020, and at 1 in TROE2021. For the latter, object 0 is the envelop.

In [ ]:
%%time
def EnumerateSpk(bw, TROE2020=False):
    
    # Regionprops
    labels, n_labels = label(bw, return_num = True)
    props = regionprops(labels)

    # Visualize spike number
    fig, ax = plt.subplots()
    ax.imshow(bw, cmap=plt.cm.gray)
    
    if TROE2020 == True:
        spike_ind = 1
    else:
        spike_ind = 0

    for props in props_spikes:
        y0, x0 = props.centroid
        plt.text(x0, y0, str(spike_ind), color="red", fontsize=15)
        spike_ind += 1 

def EnumerateSpkCV(bw, rgb, TextSize=5, TROE2020=False):
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(bw), connectivity=8)

    img = rgb.copy()
    counter=-1
    for c in centroids:
#         print(c)
        cx = round(c[0])
        cy = round(c[1])
        img = cv2.circle(img, (cx, cy), 10, (255, 0, 0), -1)
        img = cv2.putText(img, str(counter), (cx - 25, cy - 25),cv2.FONT_HERSHEY_SIMPLEX, TextSize, (255, 0, 0), 15)
        counter = counter+1
    plt.imshow(img)

In [ ]:
%%time
EnumerateSpk(bw1,TROE2020=False)

In [ ]:
%%time
EnumerateSpkCV(bw1, img0, TROE2020=False)
# This one is faster because doesn't require regionprops

In [ ]:
# plt.imshow(labeled_spks==1, cmap=plt.cm.gray)

## Select a spike

In [ ]:
%%time

# Select a spike from the labeled image
Selected_Spike = 13
spk = labeled_spks==Selected_Spike+1

# Crop spike
slice_x, slice_y = ndimage.find_objects(spk)[0]
cropped_spk = spk[slice_x, slice_y]
cropped_rgb = img0[slice_x, slice_y]
cropped_rgb = np.where(cropped_spk[..., None], cropped_rgb, 0)
cropped_gray = cropped_rgb @ [0.2126, 0.7152, 0.0722]
cropped_lab = color.rgb2lab(cropped_rgb)
plt.imshow(cropped_rgb)
# plt.imshow(cropped_gray)

# Plot selected spike
# plt.imshow(cropped_spk)

## Add 100 pixels to each border (optional)
# padded = np.pad(cropped_spk, ((100,100), (100,100)))
# plt.imshow(padded)

# Plot pixel distributions
## CIE Lab Color Space

In [ ]:
%%time
# L* channel (Luminosity)
PixelHist(labeled_spks, Lab, 0, 250)
# Single spike
# PixelHist(spk, Lab, 0, 250)

In [ ]:
# a* channel. Green (-a) to Red (+a)
PixelHist(labeled_spks, Lab, 1)

In [ ]:
# b* channel. Blue (-b) to Yellow (+b)
PixelHist(labeled_spks, Lab, 2, nbins = 250)

## RGB Color Space

In [ ]:
# Red channel
PixelHist(labeled_spks, img0, 0)

In [ ]:
# Green channel
PixelHist(labeled_spks, img0, 1)

In [ ]:
# Blue channel
PixelHist(labeled_spks, img0, 2)

# Detecting Length

## Define function
The function has two possible methods, the skeleton of a blurry image (`skelblur`) or the skeleton of a convexhull (`chull`). First one is more accurate and faster.

In [ ]:
def spk_length(spk, method='skelblur', Overlay=True, PlotCH=False):
    
    if method=='skelblur':
        # Severly blur the image
        blur = cv2.blur(np.float32(spk),(100,100))
        # Threshold the blur
        thrb = blur > 0.1
        skeleton = skeletonize(thrb)
#         plt.imshow(skeleton)
        
    if method=='chull':
        # Blur the image with a 50x50 kernel
        blur = cv2.blur(np.float32(spk),(50,50))

        # Get convex hull 
        chull = convex_hull_image(blur>0)

        # Perform skeletonization
        image = chull
        skeleton = skeletonize(image)
    #     plt.imshow(skeleton)
    
    # Spike length
    SpkL = cv2.countNonZero(np.float32(skeleton))
    
    if PlotCH == True:
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        ax = axes.ravel()

        ax[0].set_title('Original picture')
        ax[0].imshow(spk, cmap=plt.cm.gray)
        ax[0].set_axis_off()

        ax[1].set_title('Transformed picture')
        ax[1].imshow(chull, cmap=plt.cm.gray)
        ax[1].set_axis_off()

        plt.tight_layout()
        plt.show()
    
    # Visualize overlay?
    if Overlay == True:
        overlay_images = cv2.addWeighted(np.float32(spk),20,np.float32(skeleton),255,0)
        plt.imshow(overlay_images, cmap='gray')
    
    return SpkL
    
    

## Execution on selected spike

In [ ]:
%%time
spkl = spk_length(cropped_spk, Overlay=True)
print('\n', 'Approximated spike length =',spkl, 'pixels', '\n')

# Skeletonize

## Resize

In [ ]:
%%time
rescaled_spk = rescale(cropped_gray, 0.1, preserve_range=True, multichannel=False, anti_aliasing=True)
# plt.imshow(rescaled_spk)
# np.amax(rescaled_spk)

# Image.resize(size, resample=None, box=None, reducing_gap=None)

# Reduce image size
im = Image.fromarray((cropped_gray).astype(np.uint8))
(width, height) = (im.width // 30, im.height // 30)
rescaled_spk2 = im.resize((width, height))

# Increase to original size
(width, height) = (im.width, im.height)
rescaled_spk2 = rescaled_spk2.resize((width, height))
plt.imshow(rescaled_spk2)
rescaled_spk2 = np.asarray(rescaled_spk2)

# rescaled_spk2 = Image.fromarray((rescaled_spk).astype(np.uint8))
# rescaled_spk2 = rescaled_spk2.resize((cropped_rgb.shape[1],cropped_rgb.shape[0]))
# # plt.imshow(rescaled_spk2)
# # rescaled_spk2.size
# opening = np.asarray(rescaled_spk2)

## Equalization and blurring

In [ ]:
# Histogram equalization
rescaled_spk3 = exposure.equalize_hist(rescaled_spk2)

# Blur with a Gaussian
blurred = filters.gaussian(rescaled_spk3, sigma=1, preserve_range=True)

# Adaptative equalization
blurred = exposure.equalize_adapthist(blurred)

# Normalize
blurred = cv2.normalize(blurred, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
blurred = blurred.astype(np.uint8)

plt.imshow(blurred)

## Thresholding

In [ ]:
# Threshold at 80%
ret, thresh = cv2.threshold(blurred, 0.8*255, 255, 0)
thresh = np.uint8(thresh)

nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
sizes = stats[1:, -1]; nb_components = nb_components - 1    

thresh2 = np.zeros((output.shape))
#for every component in the image, you keep it only if it's above min_size
for i in range(0, nb_components):
    if sizes[i] >= MinSize:
        thresh2[output == i + 1] = 255


thresh2 = np.uint8(thresh2)
plt.imshow(thresh2)

## Contours

In [ ]:
def SpkContours(cropped_rgb, ResizeFactor=30, MinSize = 1000, plot=True):
    
    # Copy iamge
    OutImage = cropped_rgb.copy()
    
    # Convert to gray
    cropped_gray = color.rgb2gray(OutImage)
    
    # Reduce image size
    im = Image.fromarray((cropped_gray*255).astype(np.uint8))
    (width, height) = (im.width // ResizeFactor, im.height // ResizeFactor)
    rescaled_spk = im.resize((width, height))

    # Increase to original size
    (width, height) = (im.width, im.height)
    rescaled_spk = rescaled_spk.resize((width, height))
    rescaled_spk = np.asarray(rescaled_spk)

    # Histogram equalization
    rescaled_spk = exposure.equalize_hist(rescaled_spk)

    # Blur with a Gaussian
    blurred = filters.gaussian(rescaled_spk, sigma=1, preserve_range=True)

    # Adaptative equalization
    blurred = exposure.equalize_adapthist(blurred)

    # Normalize
    blurred = cv2.normalize(blurred, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    blurred = blurred.astype(np.uint8)

    # Find contours at a constant value of 0.8
    
    # Threshold at 80%
    ret, thresh = cv2.threshold(blurred, 0.8*255, 255, 0)
    thresh = np.uint8(thresh)
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1    
       
    thresh2 = np.zeros((output.shape))
    #for every component in the image, you keep it only if it's above min_size
    for i in range(0, nb_components):
        if sizes[i] >= MinSize:
            thresh2[output == i + 1] = 255
    
#     plt.imshow(thresh2)
    thresh2 = np.uint8(thresh2)
    
    EnumerateSpk(thresh2, OutImage, TROE2020=False)
    
#     contours = measure.find_contours(blurred,(0.8*255))
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     len(contours)
    
    # Detected spikelets
    print("Detected spikeletes: ", len(contours))
    
    if plot==True:
        img = OutImage.copy()
        # Plot all found contours
        plot_contours = cv2.drawContours(img, contours, -1, (0,255,0), 10)
        plt.imshow(plot_contours)


In [ ]:
%%time
SpkContours(cropped_rgb, MinSize = 1000)

## Spikelet properties

In [ ]:
def SpkltProps(cropped_rgb, img_name, ResizeFactor=30, MinSize = 1000, plot=True):
    
    # Copy image
    OutImage = cropped_rgb.copy()
    
    OutImageLab = color.rgb2lab(OutImage)
    
    # Convert to gray
    cropped_gray = color.rgb2gray(OutImage)
    
    # Reduce image size
    im = Image.fromarray((cropped_gray*255).astype(np.uint8))
    (width, height) = (im.width // ResizeFactor, im.height // ResizeFactor)
    rescaled_spk = im.resize((width, height))

    # Increase to original size
    (width, height) = (im.width, im.height)
    rescaled_spk = rescaled_spk.resize((width, height))
    rescaled_spk = np.asarray(rescaled_spk)

    # Histogram equalization
    rescaled_spk = exposure.equalize_hist(rescaled_spk)

    # Blur with a Gaussian
    blurred = filters.gaussian(rescaled_spk, sigma=1, preserve_range=True)

    # Adaptative equalization
    blurred = exposure.equalize_adapthist(blurred)

    # Normalize
    blurred = cv2.normalize(blurred, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    blurred = blurred.astype(np.uint8)

    # Find contours at a constant value of 0.8
    # Threshold at 80%
    ret, thresh = cv2.threshold(blurred, 0.8*255, 255, 0)
    thresh = np.uint8(thresh)
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1
# plt.imshow(output==1)
#     labels = output
       
    thresh2 = np.zeros((output.shape))
    #for every component in the image, you keep it only if it's above min_size
    for i in range(0, nb_components):
        if sizes[i] >= MinSize:
            thresh2[output == i + 1] = 255
    
#     plt.imshow(thresh2)
    thresh2 = np.uint8(thresh2)
    
    #     contours = measure.find_contours(blurred,(0.8*255))
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     len(contours)
    
    Props = ObjProps(thresh2, OutImage, OutImageLab, img_name='some_name')

    
    # Remove contours smallers than MinSize
#     Filtered = [c for c in contours if cv2.contourArea(c) > MinSize]
    
    # Detected spikelets
    print("Detected spikeletes: ", len(contours))
    
    # Create list for slopes
    Slopes = []
#     len(Slopes)

    if plot==True:

        # Plot all found contours
        OutImage = cv2.drawContours(OutImage, contours, -1, (255,255,255), 10);
        
        # Palette
#         cmap = matplotlib.cm.get_cmap('Spectral')
#         color_n = 0.1
        
        for c in contours:
            
            # Generate random color from palette
#             random_channels = cmap(color_n)
            random_channels = (np.random.choice(range(256), size=3))
            rr = int(random_channels[0])
            rg = int(random_channels[1])
            rb = int(random_channels[2])
            
            ellipse = cv2.fitEllipse(c)
            OutImage = cv2.ellipse(OutImage,ellipse,(rr,rg,rb),10);

            # Fit a line 
            rows,cols = OutImage.shape[:2]
            [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01);
            lefty = int((-x*vy/vx) + y)
            righty = int(((cols-x)*vy/vx)+y)
            OutImage = cv2.line(OutImage,(cols-1,righty),(0,lefty),(rr,rg,rb),3);
            
            # Slope from tope left, which is is the origin [0,0]
            rise = (0,lefty)[1] - (cols-1,righty)[1]
            run = cols
            Slope = rise/run
            Slopes.append(Slope)
            
#             color_n = color_n+0.015
            
        
        # Plot
        plt.imshow(OutImage)
        
    else:
        
        for c in contours:
            
            ellipse = cv2.fitEllipse(c)
            
            # Fit a line 
            rows,cols = OutImage.shape[:2]
            [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01);
            lefty = int((-x*vy/vx) + y)
            righty = int(((cols-x)*vy/vx)+y)

            rise = (0,lefty)[1] - (cols-1,righty)[1]
            run = cols
            Slope = rise/run
            Slopes.append(Slope)
    
    # Add slopes to data frame
    Props['Spklt_Angle'] = Slopes
    
    return Props

In [ ]:
%%time
SP = SpkltProps(cropped_rgb, img_name=Images[0])
# SP


## Contour props

In [ ]:
def ObjProps(mask, cropped_rgb, cropped_lab, img_name):

    # Regionprops
    labeled_contours, num_contours = label(mask, return_num = True)
    props_contours = regionprops(labeled_contours)

    # # Create column with image name
    Image_Name = img_name.split('\\')[-1]
    Image_Name = [Image_Name] * num_contours

    # Geometric properties
    Labels = [rp.label for rp in props_contours]
    Areas = [rp.area for rp in props_contours]
    MajorAxes = [rp.major_axis_length for rp in props_contours]
    MinorAxes = [rp.minor_axis_length for rp in props_contours]
    Orientations = [rp.orientation for rp in props_contours]
    Perimeters = [rp.perimeter for rp in props_contours]
    Eccentricities = [rp.eccentricity for rp in props_contours]

    # Spectral properties
    red_props = regionprops(labeled_contours, intensity_image=cropped_rgb[:,:,0])
    green_props = regionprops(labeled_contours, intensity_image=cropped_rgb[:,:,1])
    blue_props = regionprops(labeled_contours, intensity_image=cropped_rgb[:,:,2])
    L_props = regionprops(labeled_contours, intensity_image=cropped_lab[:,:,0])
    a_props = regionprops(labeled_contours, intensity_image=cropped_lab[:,:,1])
    b_props = regionprops(labeled_contours, intensity_image=cropped_lab[:,:,2])

    red = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in red_props])
    green = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in green_props])
    blue = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in blue_props])
    L = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in L_props])
    a = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in a_props])
    b = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in b_props])

    Red_Perc = np.array(channel_percentiles(red_props, Negatives=False)).T
    Green_Perc = np.array(channel_percentiles(green_props, Negatives=False)).T
    Blue_Perc = np.array(channel_percentiles(blue_props, Negatives=False)).T
    L_Perc = np.array(channel_percentiles(L_props)).T
    a_Perc = np.array(channel_percentiles(a_props, Negatives=True)).T
    b_Perc = np.array(channel_percentiles(b_props, Negatives=True)).T

    # Dataframe 1: for single obervation per spike
    contours_per_image = pd.DataFrame(
    list(zip(Image_Name, Labels, Areas, MajorAxes, MinorAxes, Orientations, Eccentricities, Perimeters, 
    red[:,0], red[:,1], red[:,2], green[:,0], green[:,1], green[:,2], blue[:,0], blue[:,1], blue[:,2], 
    L[:,0], L[:,1], L[:,2], a[:,0], a[:,1], a[:,2], b[:,0], b[:,1], b[:,2], 
    Red_Perc[:,0], Red_Perc[:,1], Red_Perc[:,2], Red_Perc[:,3], Red_Perc[:,4], Red_Perc[:,5], Red_Perc[:,6], 
    Green_Perc[:,0], Green_Perc[:,1], Green_Perc[:,2], Green_Perc[:,3], Green_Perc[:,4], Green_Perc[:,5], Green_Perc[:,6], 
    Blue_Perc[:,0], Blue_Perc[:,1], Blue_Perc[:,2], Blue_Perc[:,3], Blue_Perc[:,4], Blue_Perc[:,5], Blue_Perc[:,6], 
    L_Perc[:,0], L_Perc[:,1], L_Perc[:,2], L_Perc[:,3], L_Perc[:,4], L_Perc[:,5], L_Perc[:,6], 
    a_Perc[:,0], a_Perc[:,1], a_Perc[:,2], a_Perc[:,3], a_Perc[:,4], a_Perc[:,5], a_Perc[:,6], 
    a_Perc[:,7], a_Perc[:,8], a_Perc[:,9], a_Perc[:,10], a_Perc[:,11], a_Perc[:,12], a_Perc[:,13], 
    b_Perc[:,0], b_Perc[:,1], b_Perc[:,2], b_Perc[:,3], b_Perc[:,4], b_Perc[:,5], b_Perc[:,6], 
    b_Perc[:,7], b_Perc[:,8], b_Perc[:,9], b_Perc[:,10], b_Perc[:,11], b_Perc[:,12], b_Perc[:,13])), 
    columns = ['Image_Name', 'Spklt_Label', 'Area', 'MajorAxis', 'MinorAxes', 'Orientation', 'Eccentricity', 'Perimeter', 
      'Red_mean', 'Red_min', 'Red_max', 'Green_mean', 'Green_min', 'Green_max', 'Blue_mean', 'Blue_min', 'Blue_max', 
      'L_mean', 'L_min', 'L_max', 'a_mean', 'a_min', 'a_max', 'b_mean', 'b_min', 'b_max', 
      'Red_p25', 'Red_p50', 'Red_p75', 'Red_Mean', 'Red_sd', 'Red_Min', 'Red_Max', 
      'Green_p25', 'Green_p50', 'Green_p75', 'Green_Mean', 'Green_sd', 'Green_Min', 'Green_Max', 
      'Blue_p25', 'Blue_p50', 'Blue_p75', 'Blue_Mean', 'Blue_sd', 'Blue_Min', 'Blue_Max', 
      'L_p25', 'L_p50', 'L_p75', 'L_Mean', 'L_sd', 'L_Min', 'L_Max', 
      'a_p25_pos', 'a_p50_pos', 'a_p75_pos', 'a_Mean_pos', 'a_sd_pos', 'a_Min_pos', 'a_Max_pos', 
      'a_p25_neg', 'a_p50_neg', 'a_p75_neg', 'a_Mean_neg', 'a_sd_neg', 'a_Min_neg', 'a_Max_neg', 
      'b_p25_pos', 'b_p50_pos', 'b_p75_pos', 'b_Mean_pos', 'b_sd_pos', 'b_Min_pos', 'b_Max_pos', 
      'b_p25_neg', 'b_p50_neg', 'b_p75_neg', 'b_Mean_neg', 'b_sd_neg', 'b_Min_neg', 'b_Max_neg'])

    contours_per_image['Circularity'] = (4 * np.pi * contours_per_image['Area']) / (contours_per_image['Perimeter'] ** 2)

    return contours_per_image


In [ ]:
%%time
ObjP = ObjProps(mask = thresh2, cropped_rgb=cropped_rgb, cropped_lab=cropped_lab, img_name=Images[0])
ObjP

In [ ]:
EnumerateSpkCV(thresh2, cropped_rgb, TextSize=3,  TROE2020=False)

## Angles

In [ ]:
L = np.array([1,2,3,4])
L = L.reshape(len(L), 1)
LT = L.T
matL = np.multiply(L, LT)
len(matL)

In [ ]:
np.cov(matL)

In [ ]:
# S = np.array(Slopes)
# S = S.reshape(len(S), 1)
# ST = S.T
# matS = np.multiply(S, ST)
# matS.shape



def Heatmat(a, frames=30):
    a = np.array(a)
    a = a.reshape(len(a), 1)
    aT = a.T
    mat = np.multiply(a, aT)
#     mata.shape

    for frame in range(frames):
        if frame < 10:
            name = "./GIFS/Angles_0"+str(frame)+".png"
        else:
            name = "./GIFS/Angles_"+str(frame)+".png"
        
        new_mat = np.log10( 1+(mat**(frame) ) )
        sns.heatmap(new_mat)
        plt.savefig(name)
        plt.close()

In [ ]:
%%time
S = np.array(Slopes)
S = S.reshape(len(S), 1)
Heatmat(S, frames=30)

## GIFs

In [ ]:
filenames = glob.glob("./GIFS/" + '*.png', recursive=False)

In [ ]:
%%time
with imageio.get_writer('./GIFS/GIF_Angles.gif', mode='I', duration=0.25) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
%%time
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('./GIFS/GIF_Angles2.gif', images, duration=0.25)

## Euclidean distances of centroids
**NOTE:** Curvature of the spike must be consider as it alters the real distance between spikelets.

In [ ]:
nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(thresh2), 
                                                                           connectivity=8)
# len(centroids[1:][:])
img_center = centroids[0][:]
c_points = centroids[1:][:]
c_df = pd.DataFrame(c_points, columns=["x","y"])
# c_df

In [ ]:
# https://stackoverflow.com/questions/57107729/how-to-compute-multiple-euclidean-distances-of-all-points-in-a-dataset

# Consider your points as tuples in a list
data = [ (float(x),float(y)) for x,y in c_df[['x', 'y']].values ]

# Empty list to keep the distances
distances = []

for point in data:
    
    # Compute the Euclidean distance between the current point and all others
    euc_dist = [math.sqrt((point[0]-x[0] )**2 + (point[1]-x[1])**2) for x in data]
    
    # Append
    distances.append(euc_dist)

# Convert list to array
D = np.array(distances)

# Express it as a fraction from spike length
D2 = D/spkl
D2.shape


In [ ]:
# Plot distances as proportion from spike length
sns.heatmap(D2)

In [ ]:
EnumerateSpkCV(thresh2, cropped_rgb, TextSize=3,  TROE2020=False)

# Graph analysis?

In [ ]:
# https://stackoverflow.com/questions/54832694/how-to-represent-a-binary-image-as-a-graph-with-the-axis-being-height-and-width
image = blurred

#Use argmax with 200 cutoff colour in one channel
maxindex = np.argmax(image[:,:], axis=0)

# Plot graph
plt.plot(image.shape[0] - maxindex)
plt.show()

### Data

In [ ]:
data_skel = summarize(Skeleton(skel))

In [ ]:
data_skel

In [ ]:
import pytrax as pt

In [ ]:
im = ~blurred
im = np.pad(im, pad_width=50, mode='constant', constant_values=1)
plt.imshow(im)

In [ ]:
rw = pt.RandomWalk(image=im, seed=False)

In [ ]:
rw.run(nt=20000, nw=1000)
rw.plot_walk_2d()

In [ ]:
rw.plot_msd()

In [ ]:
pixel_graph, coordinates, degrees = skeleton_to_csgraph(skel)
# plt.imshow(degrees)

### Overlay

In [ ]:
draw.overlay_euclidean_skeleton_2d(rescaled_spk, data_skel, skeleton_color_source='branch-type')

In [ ]:
# Histograms
data_skel.hist(column='branch-distance', by='branch-type', bins=100)

## Medial axis

In [ ]:
# Compute the medial axis (skeleton) and the distance transform
medax, distance = medial_axis(erosion, return_distance=True)
plt.imshow(medax)

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
dilation = cv2.dilate(np.float32(medax),(5,5),iterations = 4)
plt.imshow(dilation)



In [ ]:
# plt.imshow(distance)
# distance.shape
# np.amin(distance)

In [ ]:
data_medax = summarize(Skeleton(medax))

In [ ]:
data_medax

In [ ]:
pixel_graph, coordinates, degrees = skeleton_to_csgraph(medax)
# plt.imshow(degrees)

In [ ]:
 draw.overlay_euclidean_skeleton_2d(rescaled_spk, data_medax, skeleton_color_source='branch-type')

In [ ]:
# Histograms
data_medax.hist(column='branch-distance', by='branch-type', bins=100)

# Detecting Spikelets

## Select a spike

In [ ]:
plt.imshow(cropped_rgb)

In [ ]:

def nSpklts(cropped_rgb, labeled_out=False):
    
    # Rescale to 10% of original
    rescaled_spk = rescale(cropped_rgb[...], 0.1, preserve_range=False, multichannel=True, anti_aliasing=True)
    # plt.imshow(rescaled_spk)

    # Erosion
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    erosion = cv2.erode(rescaled_spk,kernel,iterations = 1)
    # plt.imshow(erosion)

    # Opening
    kernel = np.ones((1,1),np.uint8)
    opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel, iterations = 10)
    # plt.imshow(opening)

    # Resize
    rescaled_spk2 = Image.fromarray((rescaled_spk * 255).astype(np.uint8))
    rescaled_spk2 = rescaled_spk2.resize((cropped_rgb.shape[1],cropped_rgb.shape[0]))
    # plt.imshow(rescaled_spk2)
    # rescaled_spk2.size
    opening = np.asarray(rescaled_spk2)

    # Convert rgb to gray
    gray_spklts = opening @ [0.2126, 0.7152, 0.0722]
    # plt.imshow(gray_spklts)

    # Binarize gray
    bw_spklts = gray_spklts > 0
    # plt.imshow(bw_spklts)

    # Get distances
    distance = ndi.distance_transform_edt(bw_spklts)
    # plt.imshow(-distance)

    # Get max peaks
    coords = peak_local_max(distance, min_distance=50, labels=bw_spklts)
    # plt.imshow(coords)

    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, spikelets = ndi.label(mask)
    # markers, spikelets = label(mask, return_num = True)
    # markers64 = np.int64(markers)

    if labeled_out==True:
        # Watershed
        labels = watershed(-distance, markers, mask=cropped_spk)

        # Detected spikelets
        print('Detected spikelets = ', spikelets)

        # Plot
        plt.imshow(labels, cmap=plt.cm.nipy_spectral)

    else:
        return spikelets



In [ ]:
%%time
nSpklts(cropped_rgb, labeled_out=True)

In [ ]:
rescaled_spk = rescale(cropped_rgb[...], 0.1, preserve_range=False, multichannel=True, anti_aliasing=True)
plt.imshow(rescaled_spk)


In [ ]:
# cropped_rgb.shape
# rescaled_spk2 = rescale(rescaled_spk[...], 9, preserve_range=False, multichannel=True, anti_aliasing=True)
# rescaled_spk2.shape

## Apply a Gaussian blur

In [ ]:


# kernel = np.ones((5,5),np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
erosion = cv2.erode(rescaled_spk,kernel,iterations = 1)
# plt.imshow(erosion)

kernel = np.ones((1,1),np.uint8)
opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel, iterations = 10)
plt.imshow(opening)

# blur = cv2.blur(np.float32(cropped_spk),(10,10))
# blur = cv2.blur(erosion,(30,30))   # With RGB
# plt.imshow(blur)

# kernel = np.ones((7,7),np.uint8)
# dilation = cv2.dilate(erosion,kernel,iterations = 2)
# plt.imshow(dilation)

# medfil = cv2.medianBlur(np.uint8(erosion), 5)
# plt.imshow(medfil)

# kernel = np.ones((10,10),np.uint8)
# closing = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel)
# plt.imshow(closing)




# blur = cv2.blur(erosion,(10,10))   # With RGB
# plt.imshow(blur)

## Get distances and local peaks

In [ ]:
# Resize
rescaled_spk2 = Image.fromarray((rescaled_spk * 255).astype(np.uint8))
rescaled_spk2 = rescaled_spk2.resize((cropped_rgb.shape[1],cropped_rgb.shape[0]))
# rescaled_spk2 = np.asarray(rescaled_spk2)
plt.imshow(rescaled_spk2)
rescaled_spk2.size

In [ ]:
opening = np.asarray(rescaled_spk2)
# Convert rgb to gray
gray_spklts = opening @ [0.2126, 0.7152, 0.0722]
# plt.imshow(gray_spklts)

# Binarize gray
bw_spklts = gray_spklts > 0
# plt.imshow(bw_spklts)

# Get distances
distance = ndi.distance_transform_edt(bw_spklts)
# plt.imshow(-distance)

# Get max peaks
coords = peak_local_max(distance, min_distance=50, labels=bw_spklts)
# plt.imshow(coords)

mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
markers, spikelets = ndi.label(mask)
# markers, spikelets = label(mask, return_num = True)
# markers64 = np.int64(markers)

# Watershed
labels = watershed(-distance, markers, mask=cropped_spk)

# Detected spikelets
print('Detected spikelets = ', spikelets)

## Visualize results

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(cropped_spk, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-np.float32(distance), cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()
plt.show()

In [ ]:
plt.imshow(labels==4)

In [ ]:
%%time
# Regionprops
labeled_spklts = np.int64(labels)
props_spklts = regionprops(labels)

# labeled_spks, num_spikes = label(bw1, return_num = True)
# props_spikes = regionprops(labeled_spks)

In [ ]:
spklt = labels==6
rgb_spklt = np.where(spklt[..., None], cropped_rgb, 0)
# plt.imshow(rgb_spklt)

In [ ]:
# Spikeletes Lab values
cropped_lab = color.rgb2lab(cropped_rgb)
plt.imshow(cropped_lab)
# plt.imshow(labeled_spklts, cmap='gray')
# labeled_spklts

In [ ]:
%%time

# # Create column with image name
Image_Name = Images[0].split('\\')[-1]
Image_Name = [Image_Name] * spikelets

# Geometric properties
Labels = [rp.label for rp in props_spklts]
Areas = [rp.area for rp in props_spklts]
MajorAxes = [rp.major_axis_length for rp in props_spklts]
MinorAxes = [rp.minor_axis_length for rp in props_spklts]
Orientations = [rp.orientation for rp in props_spklts]
Perimeters = [rp.perimeter for rp in props_spklts]
Eccentricities = [rp.eccentricity for rp in props_spklts]

# Spectral properties
red_props = regionprops(labeled_spklts, intensity_image=cropped_rgb[:,:,0])
green_props = regionprops(labeled_spklts, intensity_image=cropped_rgb[:,:,1])
blue_props = regionprops(labeled_spklts, intensity_image=cropped_rgb[:,:,2])
L_props = regionprops(labeled_spklts, intensity_image=cropped_lab[:,:,0])
a_props = regionprops(labeled_spklts, intensity_image=cropped_lab[:,:,1])
b_props = regionprops(labeled_spklts, intensity_image=cropped_lab[:,:,2])

red = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in red_props])
green = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in green_props])
blue = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in blue_props])
L = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in L_props])
a = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in a_props])
b = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in b_props])

Red_Perc = np.array(channel_percentiles(red_props, Negatives=False)).T
Green_Perc = np.array(channel_percentiles(green_props, Negatives=False)).T
Blue_Perc = np.array(channel_percentiles(blue_props, Negatives=False)).T
L_Perc = np.array(channel_percentiles(L_props)).T
a_Perc = np.array(channel_percentiles(a_props, Negatives=True)).T
b_Perc = np.array(channel_percentiles(b_props, Negatives=True)).T

# Dataframe 1: for single obervation per spike
Spikes_per_image = pd.DataFrame(
list(zip(Image_Name, Labels, Areas, MajorAxes, MinorAxes, Orientations, Eccentricities, Perimeters, 
red[:,0], red[:,1], red[:,2], green[:,0], green[:,1], green[:,2], blue[:,0], blue[:,1], blue[:,2], 
L[:,0], L[:,1], L[:,2], a[:,0], a[:,1], a[:,2], b[:,0], b[:,1], b[:,2], 
Red_Perc[:,0], Red_Perc[:,1], Red_Perc[:,2], Red_Perc[:,3], Red_Perc[:,4], Red_Perc[:,5], Red_Perc[:,6], 
Green_Perc[:,0], Green_Perc[:,1], Green_Perc[:,2], Green_Perc[:,3], Green_Perc[:,4], Green_Perc[:,5], Green_Perc[:,6], 
Blue_Perc[:,0], Blue_Perc[:,1], Blue_Perc[:,2], Blue_Perc[:,3], Blue_Perc[:,4], Blue_Perc[:,5], Blue_Perc[:,6], 
L_Perc[:,0], L_Perc[:,1], L_Perc[:,2], L_Perc[:,3], L_Perc[:,4], L_Perc[:,5], L_Perc[:,6], 
a_Perc[:,0], a_Perc[:,1], a_Perc[:,2], a_Perc[:,3], a_Perc[:,4], a_Perc[:,5], a_Perc[:,6], 
a_Perc[:,7], a_Perc[:,8], a_Perc[:,9], a_Perc[:,10], a_Perc[:,11], a_Perc[:,12], a_Perc[:,13], 
b_Perc[:,0], b_Perc[:,1], b_Perc[:,2], b_Perc[:,3], b_Perc[:,4], b_Perc[:,5], b_Perc[:,6], 
b_Perc[:,7], b_Perc[:,8], b_Perc[:,9], b_Perc[:,10], b_Perc[:,11], b_Perc[:,12], b_Perc[:,13])), 
columns = ['Image_Name', 'Spike_Label', 'Area', 'MajorAxis', 'MinorAxes', 'Orientation', 'Eccentricity', 'Perimeter', 
  'Red_mean', 'Red_min', 'Red_max', 'Green_mean', 'Green_min', 'Green_max', 'Blue_mean', 'Blue_min', 'Blue_max', 
  'L_mean', 'L_min', 'L_max', 'a_mean', 'a_min', 'a_max', 'b_mean', 'b_min', 'b_max', 
  'Red_p25', 'Red_p50', 'Red_p75', 'Red_Mean', 'Red_sd', 'Red_Min', 'Red_Max', 
  'Green_p25', 'Green_p50', 'Green_p75', 'Green_Mean', 'Green_sd', 'Green_Min', 'Green_Max', 
  'Blue_p25', 'Blue_p50', 'Blue_p75', 'Blue_Mean', 'Blue_sd', 'Blue_Min', 'Blue_Max', 
  'L_p25', 'L_p50', 'L_p75', 'L_Mean', 'L_sd', 'L_Min', 'L_Max', 
  'a_p25_pos', 'a_p50_pos', 'a_p75_pos', 'a_Mean_pos', 'a_sd_pos', 'a_Min_pos', 'a_Max_pos', 
  'a_p25_neg', 'a_p50_neg', 'a_p75_neg', 'a_Mean_neg', 'a_sd_neg', 'a_Min_neg', 'a_Max_neg', 
  'b_p25_pos', 'b_p50_pos', 'b_p75_pos', 'b_Mean_pos', 'b_sd_pos', 'b_Min_pos', 'b_Max_pos', 
  'b_p25_neg', 'b_p50_neg', 'b_p75_neg', 'b_Mean_neg', 'b_sd_neg', 'b_Min_neg', 'b_Max_neg'])

Spikes_per_image['Circularity'] = (4 * np.pi * Spikes_per_image['Area']) / (Spikes_per_image['Perimeter'] ** 2)


In [ ]:
plt.imshow(labeled_spklts)
plt.imshow(labeled_spks)

In [ ]:
# import csv
# labeled_spklts.to_csv('/content/drive/MyDrive/M.Sc/THESIS/ImageAnalysis/SpikeProperties/Spyk_Prop/redprops.csv')
# pwd
pd.DataFrame(green)

# Ridge detection

In [ ]:
# Crop spike
slice_x, slice_y = ndimage.find_objects(spk)[0]
cropped_spk = spk[slice_x, slice_y]
cropped_rgb = img0[slice_x, slice_y]
cropped_rgb = np.where(cropped_spk[..., None], cropped_rgb, 0)
cropped_gray = cropped_rgb @ [0.2126, 0.7152, 0.0722]
plt.imshow(cropped_rgb)
# plt.imshow(cropped_gray)

In [ ]:
%%time
blur = cv2.blur(cropped_rgb,(70,70))
# Threshold the blur
# thrb = blur > 0.2
# skeleton = skeletonize(thrb)

plt.imshow(blur)

In [ ]:
blur2 = cv2.blur(cropped_spk.astype(np.uint8),(40,40))
# blur = cv2.blur(cropped_rgb,(50,50))   # With RGB
plt.imshow(blur2)

In [ ]:
from skimage.filters import meijering, sato, frangi, hessian

In [ ]:
gray_cropped = cropped_rgb @ [0.2126, 0.7152, 0.0722]
plt.imshow(gray_cropped)

In [ ]:
blur2 = cv2.blur(np.float32(gray_cropped),(30,30))
# blur = cv2.blur(cropped_rgb,(50,50))   # With RGB
plt.imshow(blur2)

### Meijering
https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.meijering

In [ ]:
image = blur2
mei = meijering(image, sigmas=range(1, 10, 2), alpha=None, black_ridges=False, mode='reflect', cval=0)
plt.imshow(mei)


In [ ]:
image = mei > 0.2
skeleton = skeletonize(image)
plt.imshow(skeleton)

### Sato
https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.sato

In [ ]:
sat = sato(image, sigmas=range(1, 10, 2), black_ridges=False, mode='reflect', cval=0)
plt.imshow(sat)

In [ ]:
image = sat > 0.25
skeleton = skeletonize(image)
plt.imshow(skeleton)

### Frangi
https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.frangi

In [ ]:
fra =frangi(image, sigmas=range(0, 10, 2), scale_range=None, scale_step=None, black_ridges=False, mode='constant', cval=0)
plt.imshow(fra)

In [ ]:
image = fra>0
skeleton = skeletonize(image)
plt.imshow(skeleton)

### Hessian
https://scikit-image.org/docs/dev/api/skimage.filters.html#skimage.filters.hessian

In [ ]:
hes = hessian(image, sigmas=range(1, 10, 2), scale_range=None, scale_step=None, alpha=0.5, beta=0.5, gamma=15, black_ridges=True, mode='reflect', cval=0)
plt.imshow(hes)

In [ ]:
# Histogram equalization
rescaled_spk3 = exposure.equalize_hist(rescaled_spk2)
# plt.imshow(rescaled_spk3)

# median = cv2.medianBlur(rescaled_spk2, 3)
# plt.imshow(median)
blurred = filters.gaussian(rescaled_spk3, sigma=1, preserve_range=True)
blurred = exposure.equalize_adapthist(blurred)
# blurred = cv2.medianBlur(np.float32(blurred), 1)

blurred = cv2.normalize(blurred, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
blurred = blurred.astype(np.uint8)
# plt.imshow(bw)


plt.imshow(blurred)
# norm_image=blurred

# closing = cv2.morphologyEx(rescaled_spk3, cv2.MORPH_OPEN, (1,1), iterations = 10)
# plt.imshow(closing)

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1,1))
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_GRADIENT, (3,3), iterations = 3)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_BLACKHAT, (1,1), iterations = 5)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_TOPHAT, (3,3), iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_OPEN, (3,3), iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_CLOSE, (3,3), iterations = 5)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_DILATE, (3,3), iterations = 1)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_ERODE, kernel, iterations = 1)

# bw = cv2.morphologyEx(norm_image, cv2.MORPH_TOPHAT, (3,3), iterations = 10)

# blurred = filters.gaussian(blurred, sigma=0.5)

# plt.imshow(median)

# bw = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, (3,3), iterations = 5)

# bw = exposure.equalize_adapthist(bw)
# plt.imshow(rescaled_spk3)
# bw = cv2.normalize(bw, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
# bw = bw.astype(np.uint8)
# plt.imshow(bw)

# plt.imshow(blurred)
# np.min(bw)
# median = cv2.medianBlur(rescaled_spk2, 3)
# plt.imshow(median)

In [ ]:
# Histrogram
np.median(blurred)
histogram, bin_edges = np.histogram(blurred)
plt.plot(bin_edges[0:-1], histogram)

In [ ]:
ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create(dx=1, dy=1, ksize = 7)
ridges = ridge_filter.getRidgeFilteredImage(np.float32(blurred))
plt.imshow(ridges)

# Morphological operations with OpenCV
# https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html

# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
# gradient = cv2.morphologyEx(rescaled_spk2, cv2.MORPH_GRADIENT, kernel)
# plt.imshow(gradient)

# edges = cv2.Canny(cropped_rgb, 30, 100)
# plt.imshow(edges)

# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# opening = cv2.morphologyEx(ridges, cv2.MORPH_BLACKHAT, kernel, iterations = 5)
# # plt.imshow(opening)

# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# closing = cv2.morphologyEx(ridges, cv2.MORPH_OPEN, (3,3), iterations = 3)
# plt.imshow(closing)

# # kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# erode = cv2.morphologyEx(closing, cv2.MORPH_ERODE, kernel, iterations = 1)
# plt.imshow(erode)

# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
# erosion = cv2.erode(opening,kernel,iterations = 1)
# # plt.imshow(erosion)

In [ ]:
# fra=frangi(blurred, black_ridges=False, mode='constant')
# plt.imshow(fra)

# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# erosion = cv2.erode(fra,kernel,iterations = 5)
# plt.imshow(erosion)

hyst = filters.scharr_h(blurred)
plt.imshow(hyst)

# median = cv2.medianBlur(np.uint8(hyst>0), 9)
# # plt.imshow(median)

# thinned = medial_axis(rescaled_spk2[:,:,0]>0)
# plt.imshow(thinned)

# medax= medial_axis(thinned)
# plt.imshow(medax)


# median2 = cv2.medianBlur(median, 3)
# # plt.imshow(median2)

# median3 = cv2.medianBlur(median2, 3)
# plt.imshow(median3)

In [ ]:
# median = cv2.medianBlur(rescaled_spk2, 3)
# plt.imshow(median)

# ret, thresh = cv2.threshold(median, 0, 255, cv2.THRESH_BINARY)
# plt.imshow(thresh)

# contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


# Find contours at a constant value of 0.8
contours = measure.find_contours(blurred,255*0.8)

# Display the image and plot all contours found
fig, ax = plt.subplots()
ax.imshow(blurred, cmap=plt.cm.gray)

for contour in contours:
    ax.plot(contour[:, 1], contour[:, 0], linewidth=1)

ax.axis('image')
ax.set_xticks([])
ax.set_yticks([])
plt.show()

len(contours)

# # create an empty black image
# drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)

# # draw contours and hull points
# for i in range(len(contours)):
#     color_contours = (0, 255, 0) # green - color for contours
#     color = (255, 0, 0) # blue - color for convex hull
#     # draw ith contour
#     plt.drawContours(drawing, contours, i, color_contours, 1, 8, hierarchy)


In [ ]:
%%time 
# Resize
# im = Image.fromarray((cropped_gray).astype(np.uint8))
# (width, height) = (im.width // 30, im.height // 30)
# blurred2 = blurred.resize((im.width, im.height))
# blurred2 = np.asarray(blurred2)
# plt.imshow(blurred2)
# (width, height) = (im.width, im.height)
# rescaled_spk2 = rescaled_spk2.resize((width, height))

# rescaled_spk2 = np.asarray(rescaled_spk2)

skel = skeletonize(blurred>200)
plt.imshow(skel)


In [ ]:
blurred2

In [ ]:


# blur = cv2.blur(rescaled_spk2,(10,10))
# # blur = cv2.blur(erosion,(30,30))   # With RGB
# plt.imshow(blur)

kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(rescaled_spk2, cv2.MORPH_OPEN, kernel, iterations = 1)
# plt.imshow(opening)

median = cv2.medianBlur(opening, 3)
plt.imshow(median)

# kernel = np.ones((3,3),np.uint8)
# closing = cv2.morphologyEx(opening, cv2.MORPH_ERODE, kernel, iterations = 1)
# plt.imshow(closing)

In [ ]:
median2 = exposure.equalize_hist(median)
plt.imshow(median2)
# plt.imshow(color.rgb2gray(median))

In [ ]:
from skimage.draw import line

# Constructing test image
image = np.zeros((200, 200))
idx = np.arange(25, 175)
image[idx, idx] = 255
image[line(45, 25, 25, 175)] = 255
image[line(25, 135, 175, 155)] = 255

# Classic straight-line Hough transform
# Set a precision of 0.5 degree.
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)
h, theta, d = hough_line(image)

# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image)
ax[0].set_title('Input image')
ax[0].set_axis_off()

angle_step = 0.5 * np.diff(theta).mean()
d_step = 0.5 * np.diff(d).mean()
bounds = [np.rad2deg(theta[0] - angle_step),
          np.rad2deg(theta[-1] + angle_step),
          d[-1] + d_step, d[0] - d_step]
ax[1].imshow(np.log(1 + h), extent=bounds, aspect=1 / 1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
    ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))

plt.tight_layout()
plt.show()



In [ ]:
h, theta, d = hough_line_peaks(h, theta, d)

In [ ]:
medax= medial_axis(cropped_gray)
# plt.imshow(medax)

tested_angles = np.linspace(-np.pi, np.pi, 360, endpoint=False)
h, theta, d = hough_line(medax, tested_angles)
# plt.imshow(out)

# np.linspace(-np.pi / 2, np.pi / 2, 360, endpoint=False)

# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(medax)
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(np.log(1+h))
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(medax, cmap="gray")
ax[2].set_ylim((medax.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

for _, angle, dist in zip(*hough_line_peaks(h, theta, d, min_angle=0, min_distance=1)):
    (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
    ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))

plt.tight_layout()
plt.show()
# newrgb = np.where(medax[..., None], cropped_rgb, 0)
# plt.imshow(newrgb)

In [ ]:
max(theta)

In [ ]:
lines = probabilistic_hough_line(medax, line_length=10,line_gap=1)
# plt.plot(lines)

In [ ]:
# Generating figure 2
fig, axes = plt.subplots(3,1, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(cropped_gray, cmap='gray')
ax[0].set_title('Input image')

ax[1].imshow(medax, cmap='gray')
ax[1].set_title('Canny edges')

ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, cropped_gray.shape[1]))
ax[2].set_ylim((cropped_gray.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
%%time
# fra=frangi(cropped_gray, black_ridges=False, mode='constant')
# plt.imshow(fra)

norm_image = cv2.normalize(fra, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
norm_image = norm_image.astype(np.uint8)
plt.imshow(norm_image)


In [ ]:
bw = cv2.morphologyEx(norm_image, cv2.MORPH_TOPHAT, (3,3), iterations = 10)
# plt.imshow(bw)


newrgb = np.where(bw[...], cropped_gray, 0)
plt.imshow(newrgb)

In [ ]:
image = 255-newrgb
hes = hessian(image, sigmas=range(1, 10, 2), scale_range=None, scale_step=None, alpha=0.5, beta=0.5, gamma=15, black_ridges=False, mode='reflect', cval=0)
plt.imshow(hes)

In [ ]:
scipy.ndimage.morphology.distance_transform_edt

In [ ]:
ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create(dx=1, dy=1, ksize = 3)
ridges = ridge_filter.getRidgeFilteredImage(np.uint8(image))
plt.imshow(ridges)

# newrgb2 = np.where(ridges[..., None], newrgb, 0)
# plt.imshow(newrgb2)

In [ ]:
# medax= medial_axis(ridges)
# plt.imshow(medax)

norm_image = cv2.normalize(ridges, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
norm_image = norm_image.astype(np.uint8)
plt.imshow(norm_image)

In [ ]:
thin1= thin(norm_image)
plt.imshow(thin1)

In [ ]:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_GRADIENT, (3,3), iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_BLACKHAT, kernel, iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_TOPHAT, (3,3), iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_OPEN, (3,3), iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_CLOSE, (3,3), iterations = 5)
bw = cv2.morphologyEx(255-image, cv2.MORPH_DILATE, (3,3), iterations = 5)
# bw = cv2.morphologyEx(newrgb, cv2.MORPH_ERODE, (3,3), iterations = 10)
plt.imshow(bw)

# # do connected components processing
# nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(bw, None, None, None, 8, cv2.CV_32S)

# #get CC_STAT_AREA component as stats[label, COLUMN] 
# areas = stats[1:,cv2.CC_STAT_AREA]

# result = np.zeros((labels.shape), np.uint8)

# for i in range(0, nlabels - 1):
#     if areas[i] >= 100:   #keep
#         result[labels == i + 1] = 255
        
# plt.imshow(result)

In [ ]:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_GRADIENT, kernel, iterations = 5)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_BLACKHAT, kernel, iterations = 10)
# bw = cv2.morphologyEx(result, cv2.MORPH_TOPHAT, (3,3), iterations = 10)
# bw = cv2.morphologyEx(norm_image, cv2.MORPH_OPEN, kernel, iterations = 10)
# plt.imshow(bw)

# hyst = filters.scharr_h(bw)
# plt.imshow(hyst)

thinned = medial_axis(blurred>0.5)
plt.imshow(thinned)

# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# erosion = cv2.erode(cropped_lab,kernel,iterations = 5)
# # plt.imshow(erosion)

# hyst = filters.scharr_h(fra)
# plt.imshow(hyst)




# norm_image = cv2.normalize(hyst, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
# norm_image = norm_image.astype(np.uint8)
# plt.imshow(norm_image)
# thresh = np.median(hyst) + (np.std(hyst) * 2)

# medax= medial_axis(result)
# plt.imshow(medax)

In [ ]:
np.amax(medax)

In [ ]:
np.mean(hyst)

In [ ]:
# Dilation
# kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
# dilation = cv2.dilate(np.uint8(medax),kernel,iterations = 2)
# # plt.imshow(dilation)

result= thin(dilation)
plt.imshow(result)

In [ ]:

# blur = cv2.blur(cropped_gray,(50,50))
# blur = cv2.blur(opening,(3,3))   # With RGB
# plt.imshow(blur)
# np.amax(blur)
medax= medial_axis(blurred)
plt.imshow(medax)

In [ ]:
# h, theta, d = hough_line(medax)
# plt.imshow(np.uint8(medax))

# norm_image = cv2.normalize(np.uint8(medax), None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
# norm_image = norm_image.astype(np.uint8)
# plt.imshow(norm_image)

# minLineLength = 10
# maxLineGap = 5
# lines = cv2.HoughLinesP(norm_image, 1, np.pi/180, 200, minLineLength, maxLineGap)
# for x1, y1, x2, y2 in lines[0]:
#     cv2.line(norm_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
# cv2.imwrite('.\canny5.jpg', norm_image)


lines = cv2.HoughLines(norm_image,1,np.pi/180,200)
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(norm_image,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imwrite('houghlines3.jpg',norm_image)


# cv2.imwrite('.\houghlines5.jpg', cropped_rgb)

# # Generating figure 1
# fig, axes = plt.subplots(1, 3, figsize=(15, 6))
# ax = axes.ravel()

# ax[0].imshow(medax)
# ax[0].set_title('Input image')
# ax[0].set_axis_off()

# ax[1].imshow(np.log(1+h))
# ax[1].set_title('Hough transform')
# ax[1].set_xlabel('Angles (degrees)')
# ax[1].set_ylabel('Distance (pixels)')
# ax[1].axis('image')

# ax[2].imshow(medax, cmap="gray")
# ax[2].set_ylim((medax.shape[0], 0))
# ax[2].set_axis_off()
# ax[2].set_title('Detected lines')

# for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
#     (x0, y0) = dist * np.array([np.cos(angle), np.sin(angle)])
#     ax[2].axline((x0, y0), slope=np.tan(angle + np.pi/2))

# plt.tight_layout()
# plt.show()

In [ ]:
# angles

sns.histplot(h, bins=100)


In [ ]:
# Erosion
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
erosion = cv2.erode(fra,kernel,iterations =3)
plt.imshow(erosion)

erosion2 = erosion * 1000 * 255
np.amax(fra)
edges = cv2.Canny(np.uint8(fra*10000), 0.03, 0.1)
plt.imshow(edges)



# Opening
kernel = np.ones((5,5),np.uint8)
opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, (5,5), iterations = 1)
plt.imshow(opening)

blur = cv2.blur(np.float32(opening),(10,10))
blur = cv2.blur(opening,(3,3))   # With RGB
plt.imshow(blur)

# Dilation
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
dilation = cv2.dilate(blur,kernel,iterations = 2)
plt.imshow(dilation)

In [ ]:
# dilation = dilation*255
gray_skel = opening @ [0.2126, 0.7152, 0.0722]
plt.imshow(gray_skel)
np.amin(gray_skel)

## Frangi

In [ ]:
%%time
fra =frangi(cropped_gray>0, black_ridges=False)
plt.imshow(fra)

In [ ]:
np.min(cropped_gray)

## Skeleton

In [ ]:
gray_skel = gray_skel>-0.001
plt.imshow(gray_skel)

skel = skeletonize(fra>0)
plt.imshow(skel)

### Data

In [ ]:
data_skel = summarize(Skeleton(skel))

In [ ]:
data_skel

In [ ]:
pixel_graph, coordinates, degrees = skeleton_to_csgraph(skel)
# plt.imshow(degrees)

### Overlay

In [ ]:
draw.overlay_euclidean_skeleton_2d(rescaled_spk, data_skel, skeleton_color_source='branch-type')

In [ ]:
# Histograms
data_skel.hist(column='branch-distance', by='branch-type', bins=100)

## Medial axis

In [ ]:
# Compute the medial axis (skeleton) and the distance transform
medax, distance = medial_axis(erosion, return_distance=True)
plt.imshow(medax)

kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
dilation = cv2.dilate(np.float32(medax),(5,5),iterations = 4)
plt.imshow(dilation)



In [ ]:
# plt.imshow(distance)
# distance.shape
# np.amin(distance)

In [ ]:
data_medax = summarize(Skeleton(medax))

In [ ]:
data_medax

In [ ]:
pixel_graph, coordinates, degrees = skeleton_to_csgraph(medax)
# plt.imshow(degrees)

In [ ]:
 draw.overlay_euclidean_skeleton_2d(rescaled_spk, data_medax, skeleton_color_source='branch-type')

In [ ]:
# Histograms
data_medax.hist(column='branch-distance', by='branch-type', bins=100)

## Properties
Here we look at the overall properties of each spike without any detail about branches


In [ ]:
# Get geometric and spectral properties
spk_df = SpikesDF(img0_name)

In [ ]:
# Visualize column names
print(spk_df.columns)

# Functions

## `ListImages`
Returns a list of full image paths.

In [ ]:
%%time
def ListImages(path, imgformat=".tif", recursive=False):
    Images = glob.glob(path + '/*' + imgformat, recursive=True)    
    return Images

# Example:
# path = r'./Images/TEST'
# Images = ListImages(path, imgformat=".tif", recursive=False)

## `RemoveBackground`
Input can be the image as an array or just the full image path.
It takes almost 25 seconds to create list with five images (rgb, gray, lab, hsv, bw). Needs to be optimized.

In [ ]:
%%time
def RemoveBackground(img, OtsuScaling=0.25, rgb_out=True, gray_out=True, lab_out=True, hsv_out=True, bw_out=True):
    
    # Read image
    if isinstance(img, str) == True:
        img0 = plt.imread(img)
    else: 
        img0 = img
    
    # Crop images. They were all taken with the same scanner
    img1 = img0[44:6940, 25:4970, :]
    
     # Convert to gray
    gray0 = img1 @ [0.2126, 0.7152, 0.0722]
    
    # Set image threshold
    T = filters.threshold_otsu(gray0)
#     print(T)
    T = T*OtsuScaling
#     print(T)
    
    # Segment gray image
    bw0 = gray0 > T
    
    # Remove small objects
    n_pixels = gray0.shape[0] * gray0.shape[1]
    minimum_size = n_pixels/10000
    bw1 = morphology.remove_small_objects(bw0, min_size=np.floor(minimum_size))
    
    ImagesOut = []
#     len(ImagesOut)
    
    if rgb_out==True:
        # Apply mask to RGB
        rgb = np.where(bw1[..., None], img1, 0)
        ImagesOut.append(rgb)
    if gray_out==True and rgb_out==True:
        gray = color.rgb2gray(rgb)
        ImagesOut.append(gray)
    if lab_out==True and rgb_out==True:
        lab = color.rgb2lab(rgb)
        ImagesOut.append(lab)
    if hsv_out==True and rgb_out==True:
        hsv = color.rgb2hsv(rgb)
        ImagesOut.append(hsv)
    if bw_out==True:
#         # Threshold
#         otsu = filters.threshold_otsu(gray)
#         bw0 = gray > 0
#         bw = morphology.remove_small_objects(bw0, min_size=1.5e-05 * gray.shape[0] * gray.shape[1])
        ImagesOut.append(bw1)
    
    return ImagesOut

# Usage:
# %%time
# I = RemoveBackground(Images[3], OtsuScaling=0.25, rgb_out=True, gray_out=True, lab_out=True, hsv_out=True, bw_out=True)
# rgb0 = I[0]
# gray0 = I[1]
# lab0 = I[2]
# hsv0 = I[3]
# bw0 = I[4]


## `EnumerateSpkCV`
Enumerate spikes, spikelets, or contours.

In [ ]:
%%time
# Enumerate spikes
def EnumerateSpkCV(bw, rgb, TextSize=5, TROE2020=False):
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(bw), connectivity=8)
    img = rgb.copy()
    
    if TROE2020==True:
        counter=-1
    else:
        counter=0
        for c in centroids:
    #         print(c)
            cx = round(c[0])
            cy = round(c[1])
            img = cv2.circle(img, (cx, cy), 10, (255, 0, 0), -1)
            img = cv2.putText(img, str(counter), (cx - 25, cy - 25),cv2.FONT_HERSHEY_SIMPLEX, TextSize, (255, 0, 0), 15)
            counter = counter+1
        plt.imshow(img)

# # Example:
# EnumerateSpkCV(bw0, rgb0, TextSize=5, TROE2020=False)
# EnumerateSpkCV(spklts, cropped_rgb, TextSize=5, TROE2020=False)

## `spk_length`
Return the spike length

In [ ]:
%%time

def spk_length(cropped_spk, method='skelblur', Overlay=True, PlotCH=False):
    
    if method=='skelblur':
        # Severly blur the image
        blur = cv2.blur(np.float32(cropped_spk),(100,100))
        # Threshold the blur
        thrb = blur > 0.1
        skeleton = skeletonize(thrb)
#         plt.imshow(skeleton)
        
    if method=='chull':
        # Blur the image with a 50x50 kernel
        blur = cv2.blur(np.float32(cropped_spk),(50,50))

        # Get convex hull 
        chull = convex_hull_image(blur>0)

        # Perform skeletonization
        image = chull
        skeleton = skeletonize(image)
    #     plt.imshow(skeleton)
    
    # Spike length
    SpkL = cv2.countNonZero(np.float32(skeleton))
    
    if PlotCH == True:
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        ax = axes.ravel()

        ax[0].set_title('Original picture')
        ax[0].imshow(cropped_spk, cmap=plt.cm.gray)
        ax[0].set_axis_off()

        ax[1].set_title('Transformed picture')
        ax[1].imshow(chull, cmap=plt.cm.gray)
        ax[1].set_axis_off()

        plt.tight_layout()
        plt.show()
    
    # Visualize overlay?
    if Overlay == True:
        overlay_images = cv2.addWeighted(np.float32(cropped_spk),20,np.float32(skeleton),255,0)
        plt.imshow(overlay_images, cmap='gray')
    
    return SpkL

# Example:
# SL = spk_length(cropped_spk, method='skelblur', Overlay=True, PlotCH=False)

## `PixelHist`
Object histogram analysis. This function returns the black and white version of the given image(s) and print the label of each spike as computed by 'skimage.measure.label'


In [ ]:
%%time
def PixelHist(bw, ColorSpace, channel = 0, spikes="All", nbins = 100):
    
    labeled_spks, num_spikes = label(bw, return_num = True)
#     plt.imshow(labeled_spks==0)

    if spikes=="All":
        labeled_spks = labeled_spks
    else:
        for L in range(1,num_spikes+1):
#             print(L)
            if not L in spikes:
#                 print("Deleted label ", L)
                labeled_spks=np.where(labeled_spks==L, 0, labeled_spks)
#     plt.imshow(labeled_spks)
    
    Props = regionprops(labeled_spks, intensity_image=ColorSpace[:,:,channel])
    Areas = [rp.area for rp in Props]
    Labels = [rp.label for rp in Props] #Delete 1 because label in image is +1 greater than ACTUAL label
    Spikes_Data = []
    Names = []
    Colors = sns.color_palette("husl", len(spikes))
    Colors2 = [list(i) for i in Colors] # list of lists
    
    
    for indexed in range(len(Labels)):        
        spk_data = Props[indexed].intensity_image 
        spk_data = spk_data.ravel()
        NonZero = spk_data[spk_data != 0]

        Spikes_Data.append(NonZero)
        Names.append("Spike " + str(int(spikes[indexed])) + "\n" + "Area = "  + str(round(np.mean(NonZero))) + " px" + "\n" +
                     "Mean = "  + str(round(np.mean(NonZero), 1)))
        Colors.append(list(np.random.choice(range(2), size=3)))
    
    plt.hist(Spikes_Data, bins = nbins, color = Colors2, label = Names);
    
    # Plot formatting
    plt.legend();
    plt.xlabel('Intensity Value');
    plt.ylabel('Number of NonZero Pixels');
    plt.title('Distribution of None-Zero Pixel Values for Selected Given Channel and Spikes');

# Example:
# PixelHist(bw=bw0, ColorSpace=lab0, channel = 0, spikes=[1,2,26], nbins = 100)

## `channel_percentiles`
This returns the percentiles for a given channel.

In [ ]:
%%time
def channel_percentiles(channel_props, Negatives = None):
      
    # Create empty lists to populate    
    p25_pos_list = []
    p50_pos_list = []
    p75_pos_list = []
    p25_neg_list = []
    p50_neg_list = []
    p75_neg_list = []
    mean_pos_list = []
    sd_pos_list = []
    min_pos_list = []
    max_pos_list = []
    mean_neg_list = []
    sd_neg_list = []
    min_neg_list = []
    max_neg_list = []
    
#     channel_props[1].intensity_image
    

    # channel_props = a_props
    for spk in range(len(channel_props)):
        # spk=0
        my_array = channel_props[spk].intensity_image
#         plt.imshow(my_array)
        flat_array = my_array.ravel()
        non_zero = flat_array[flat_array != 0]

        positive_values = non_zero[non_zero > 0]
        if positive_values.size == 0:
            positive_values = [0]        
        p25_pos = np.nanpercentile(positive_values, 25)
        p25_pos_list.append(p25_pos)
        p50_pos = np.nanpercentile(positive_values, 50)
        p50_pos_list.append(p50_pos)
        p75_pos = np.nanpercentile(positive_values, 75)
        p75_pos_list.append(p75_pos)
        mean_pos = np.mean(positive_values)
        mean_pos_list.append(mean_pos)
        sd_pos = np.std(positive_values)
        sd_pos_list.append(sd_pos)
        min_pos = min(positive_values)
        min_pos_list.append(min_pos)
        max_pos = max(positive_values)
        max_pos_list.append(max_pos)

        if Negatives == True:
            negative_values = non_zero[non_zero < 0]
            # Make sure list is not empty, otherwise add a zero
            if negative_values.size == 0:
                negative_values = [0]
            p25_neg = np.nanpercentile(negative_values, 25)
            p25_neg_list.append(p25_neg)
            p50_neg = np.nanpercentile(negative_values, 50)
            p50_neg_list.append(p50_neg)
            p75_neg = np.nanpercentile(negative_values, 75)
            p75_neg_list.append(p75_neg)
            mean_neg = np.mean(negative_values)
            mean_neg_list.append(mean_neg)
            sd_neg = np.std(positive_values)
            sd_neg_list.append(sd_neg)
            min_neg = min(negative_values)
            min_neg_list.append(min_neg)
            max_neg = max(negative_values)
            max_neg_list.append(max_neg)

            Lists = [p25_pos_list, p50_pos_list, p75_pos_list, mean_pos_list, sd_pos_list, min_pos_list, max_pos_list, p25_neg_list, p50_neg_list, p75_neg_list, mean_neg_list, sd_neg_list, min_neg_list, max_neg_list]
        else:
            Lists = [p25_pos_list, p50_pos_list, p75_pos_list, mean_pos_list, sd_pos_list, min_pos_list, max_pos_list]

        
        # if any(pixel < 0 for pixel in non_zero) == True:
        #     negative_values = non_zero[non_zero < 0]
        #     p25_neg = np.percentile(negative_values, 25)
        #     p25_neg_list.append(p25_neg)
        #     p50_neg = np.percentile(negative_values, 50)
        #     p50_neg_list.append(p50_neg)
        #     p75_neg = np.percentile(negative_values, 75)
        #     p75_neg_list.append(p75_neg)
    return Lists


########################################################################################
# Examples:

# For one spike
# labeled_contours, num_contours = label(cropped_spk, return_num = True)
# red_props = regionprops(labeled_contours, intensity_image=cropped_rgb)
# a_Perc = np.array(channel_percentiles(channel_props=a_props, Negatives=True)).T
# len(a_Perc)


# For one entire image
# labeled_contours, num_contours = label(bw0, return_num = True)
# red_props = regionprops(labeled_contours, intensity_image=rgb0)
# Red_Perc = np.array(channel_percentiles(channel_props=red_props, Negatives=False)).T
# len(Red_Perc)

## `SpikesDF`
Spike properties in a DataFrame. This function returns a Pandas data frame with the geometric and spectral properties of the given path to rgb image

In [ ]:
%%time
def SpikesDF(I, ImagePath, RemoveBG=False, PrintSpkLabels=False, rm_envelope=False):
    
    # Check if images or path were given
    if RemoveBG == True:
        # Remove background (path was given)
        I = RemoveBackground(ImagePath, OtsuScaling=0.25, rgb_out=True, gray_out=True, lab_out=True, hsv_out=True, bw_out=True)
        rgb0 = I[0]
        gray0 = I[1]
        lab0 = I[2]
        hsv0 = I[3]
        bw0 = I[4]
        
        Image_Name = ImagePath.split('\\')[-1]

    else: 
        # Images were given in a list as returned by RemoveBackground()
        rgb0 = I[0]
        gray0 = I[1]
        lab0 = I[2]
        hsv0 = I[3]
        bw0 = I[4]
    
    
    labeled_spks, num_spikes = label(bw0, return_num = True)
    props_spikes = regionprops(labeled_spks)
    
    # Create column with image name
    Image_Name = ImagePath.split('\\')[-1]
    Image_Name = [Image_Name] * num_spikes
    
    # Geometric properties
    Labels = [rp.label for rp in props_spikes]
    Areas = [rp.area for rp in props_spikes]
    MajorAxes = [rp.major_axis_length for rp in props_spikes]
    MinorAxes = [rp.minor_axis_length for rp in props_spikes]
    Orientations = [rp.orientation for rp in props_spikes]
    Perimeters = [rp.perimeter for rp in props_spikes]
    Eccentricities = [rp.eccentricity for rp in props_spikes]
   
    # Spectral properties
    red_props = regionprops(labeled_spks, intensity_image=rgb0[:,:,0])
    green_props = regionprops(labeled_spks, intensity_image=rgb0[:,:,1])
    blue_props = regionprops(labeled_spks, intensity_image=rgb0[:,:,2])
    L_props = regionprops(labeled_spks, intensity_image=lab0[:,:,0])
    a_props = regionprops(labeled_spks, intensity_image=lab0[:,:,1])
    b_props = regionprops(labeled_spks, intensity_image=lab0[:,:,2])
    H_props = regionprops(labeled_spks, intensity_image=hsv0[:,:,0])
    S_props = regionprops(labeled_spks, intensity_image=hsv0[:,:,1])
    V_props = regionprops(labeled_spks, intensity_image=hsv0[:,:,2])
    
    red = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in red_props])
    green = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in green_props])
    blue = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in blue_props])
    L = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in L_props])
    a = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in a_props])
    b = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in b_props])
    H = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in H_props])
    S = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in S_props])
    V = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in V_props])
    
    Red_Perc = np.array(channel_percentiles(red_props, Negatives=False)).T
    Green_Perc = np.array(channel_percentiles(green_props, Negatives=False)).T
    Blue_Perc = np.array(channel_percentiles(blue_props, Negatives=False)).T
    L_Perc = np.array(channel_percentiles(L_props)).T
    a_Perc = np.array(channel_percentiles(a_props, Negatives=True)).T
    b_Perc = np.array(channel_percentiles(b_props, Negatives=True)).T
    H_Perc = np.array(channel_percentiles(H_props)).T
    S_Perc = np.array(channel_percentiles(S_props)).T
    V_Perc = np.array(channel_percentiles(V_props)).T

    # Dataframe 1: for single obervation per spike
    Spikes_per_image = pd.DataFrame(
    list(zip(Image_Name, Labels, Areas, MajorAxes, MinorAxes, Orientations, Eccentricities, Perimeters, 
             red[:,0], red[:,1], red[:,2], green[:,0], green[:,1], green[:,2], blue[:,0], blue[:,1], blue[:,2], 
             L[:,0], L[:,1], L[:,2], a[:,0], a[:,1], a[:,2], b[:,0], b[:,1], b[:,2], 
             H[:,0], H[:,1], H[:,2], S[:,0], S[:,1], S[:,2], V[:,0], V[:,1], V[:,2], 
             Red_Perc[:,0], Red_Perc[:,1], Red_Perc[:,2], Red_Perc[:,3], Red_Perc[:,4], Red_Perc[:,5], Red_Perc[:,6], 
             Green_Perc[:,0], Green_Perc[:,1], Green_Perc[:,2], Green_Perc[:,3], Green_Perc[:,4], Green_Perc[:,5], Green_Perc[:,6], 
             Blue_Perc[:,0], Blue_Perc[:,1], Blue_Perc[:,2], Blue_Perc[:,3], Blue_Perc[:,4], Blue_Perc[:,5], Blue_Perc[:,6], 
             L_Perc[:,0], L_Perc[:,1], L_Perc[:,2], L_Perc[:,3], L_Perc[:,4], L_Perc[:,5], L_Perc[:,6], 
             a_Perc[:,0], a_Perc[:,1], a_Perc[:,2], a_Perc[:,3], a_Perc[:,4], a_Perc[:,5], a_Perc[:,6], 
             a_Perc[:,7], a_Perc[:,8], a_Perc[:,9], a_Perc[:,10], a_Perc[:,11], a_Perc[:,12], a_Perc[:,13], 
             b_Perc[:,0], b_Perc[:,1], b_Perc[:,2], b_Perc[:,3], b_Perc[:,4], b_Perc[:,5], b_Perc[:,6], 
             b_Perc[:,7], b_Perc[:,8], b_Perc[:,9], b_Perc[:,10], b_Perc[:,11], b_Perc[:,12], b_Perc[:,13],
             H_Perc[:,0], H_Perc[:,1], H_Perc[:,2], H_Perc[:,3], H_Perc[:,4], H_Perc[:,5], H_Perc[:,6],
             S_Perc[:,0], S_Perc[:,1], S_Perc[:,2], S_Perc[:,3], S_Perc[:,4], S_Perc[:,5], S_Perc[:,6],
             V_Perc[:,0], V_Perc[:,1], V_Perc[:,2], V_Perc[:,3], V_Perc[:,4], V_Perc[:,5], V_Perc[:,6])), 
    columns = ['Image_Name', 'Spike_Label', 'Area', 'MajorAxis', 'MinorAxes', 'Orientation', 'Eccentricity', 'Perimeter', 
               'Red_mean', 'Red_min', 'Red_max', 'Green_mean', 'Green_min', 'Green_max', 'Blue_mean', 'Blue_min', 'Blue_max', 
               'L_mean', 'L_min', 'L_max', 'a_mean', 'a_min', 'a_max', 'b_mean', 'b_min', 'b_max',
               'H_mean', 'H_min', 'H_max', 'S_mean', 'S_min', 'S_max', 'V_mean', 'V_min', 'V_max', 
               'Red_p25', 'Red_p50', 'Red_p75', 'Red_Mean', 'Red_sd', 'Red_Min', 'Red_Max', 
               'Green_p25', 'Green_p50', 'Green_p75', 'Green_Mean', 'Green_sd', 'Green_Min', 'Green_Max', 
               'Blue_p25', 'Blue_p50', 'Blue_p75', 'Blue_Mean', 'Blue_sd', 'Blue_Min', 'Blue_Max', 
               'L_p25', 'L_p50', 'L_p75', 'L_Mean', 'L_sd', 'L_Min', 'L_Max', 
               'a_p25_pos', 'a_p50_pos', 'a_p75_pos', 'a_Mean_pos', 'a_sd_pos', 'a_Min_pos', 'a_Max_pos', 
               'a_p25_neg', 'a_p50_neg', 'a_p75_neg', 'a_Mean_neg', 'a_sd_neg', 'a_Min_neg', 'a_Max_neg', 
               'b_p25_pos', 'b_p50_pos', 'b_p75_pos', 'b_Mean_pos', 'b_sd_pos', 'b_Min_pos', 'b_Max_pos', 
               'b_p25_neg', 'b_p50_neg', 'b_p75_neg', 'b_Mean_neg', 'b_sd_neg', 'b_Min_neg', 'b_Max_neg',
               'H_p25', 'H_p50', 'H_p75', 'H_Mean', 'H_sd', 'H_Min', 'H_Max',
               'S_p25', 'S_p50', 'S_p75', 'S_Mean', 'S_sd', 'S_Min', 'S_Max',
               'V_p25', 'V_p50', 'V_p75', 'V_Mean', 'V_sd', 'V_Min', 'V_Max'])
    
    Spikes_per_image['Circularity'] = (4 * np.pi * Spikes_per_image['Area']) / (Spikes_per_image['Perimeter'] ** 2)
    
    # Remove envelope's data  
    if rm_envelope==True:
        return Spikes_per_image.iloc[1: , :]
    else:
        return Spikes_per_image


# Example:
# df = SpikesDF(I, ImagePath=img_name, RemoveBG=False, PrintSpkLabels=False)

## `SpkltThresh`
Threshold for contour detection

In [ ]:
%%time

def SpkltThresh(cropped, ResizeFactor=30, thr2=0.8, MinSize=1000):   
    
    # Check that it's a gray image
    if len(cropped_rgb.shape) > 2:
        # Convert to gray
        cropped_gray = color.rgb2gray(cropped)
    else:
        cropped_gray = cropped
        
   
    # Reduce image size
    im = Image.fromarray((cropped_gray*255).astype(np.uint8))
    (width, height) = (im.width // ResizeFactor, im.height // ResizeFactor)
    rescaled_spk = im.resize((width, height))

    # Increase to original size
    (width, height) = (im.width, im.height)
    rescaled_spk = rescaled_spk.resize((width, height))
    rescaled_spk = np.asarray(rescaled_spk)

    # Histogram equalization
    rescaled_spk = exposure.equalize_hist(rescaled_spk)

    # Blur with a Gaussian
    blurred = filters.gaussian(rescaled_spk, sigma=1, preserve_range=True)

    # Adaptative equalization
    blurred = exposure.equalize_adapthist(blurred)

    # Normalize
    blurred = cv2.normalize(blurred, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    blurred = blurred.astype(np.uint8)
    
    if thr2 < 1 == True:
        thr2 = thr2*255
    else:
        thr2 = thr2
    
    # Threshold at given %
    ret, thresh = cv2.threshold(blurred, thr2, 255, 0)
    thresh = np.uint8(thresh)
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
    sizes = stats[1:, -1]; nb_components = nb_components - 1    
       
    thresh2 = np.zeros((output.shape))
    
    # Keep only objects with minimum size
    for i in range(0, nb_components):
        if sizes[i] >= MinSize:
            thresh2[output == i + 1] = 255
    
#     plt.imshow(thresh2)
    thresh2 = np.uint8(thresh2)
    
    return thresh2

# Example:
# thresh2 = SpkltThresh(cropped=cropped_rgb, ResizeFactor=30, thr2=0.8, MinSize=1000)
# plt.imshow(thresh2)


## `LabelContours`
Spike's contours

In [ ]:
%%time

# Spike's contours
def LabelContours(cropped_rgb, thresh2, ResizeFactor=30, MinSize = 1000, plot=True, thr2=0.8):
    
    # Copy iamge
    OutImage = cropped_rgb.copy()
    
    if thresh2 is not None:
        thresh2 = thresh2
    else:
        # Threshold for contours
        thresh2 = SpkltThresh(cropped=OutImage, ResizeFactor=ResizeFactor, thr2=thr2, MinSize=MinSize)

#     # Threshold for contours
#     thresh2 = SpkltThresh(cropped=cropped_rgb, ResizeFactor=30, thr2=thr2, MinSize=MinSize)
#     plt.imshow(bw_cont)
    
    # Enumerate objects
    # EnumerateSpkCV(thresh2, OutImage, TextSize=5, TROE2020=False)
    
#     contours = measure.find_contours(blurred,(0.8*255))
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     len(contours)
    
    # Detected spikelets
    print("Detected spikeletes: ", len(contours))
    
    if plot==True:
        img = OutImage.copy()
        # Plot all found contours
        plot_contours = cv2.drawContours(img, contours, -1, (0,255,0), 10)
        plt.imshow(plot_contours)
    
    return contours

# Example:
# labels_cont = LabelContours(cropped_rgb, thresh2=thresh2, ResizeFactor=30, MinSize = 1000, thr2=0.8, plot=True)
# len(labels_cont)

## `ObjProps`
Object properties. A general function to get properties of labels ROIs.

In [ ]:
%%time

def ObjProps(labeled, cropped_rgb, cropped_lab, cropped_hsv, ImagePath, MinSize = 1000, rm_envelope=False):
    
    # Label + regionprops
    labeled_contours, num_contours = label(labeled, return_num = True)
    props_contours = regionprops(labeled_contours)
#     plt.imshow(labeled_contours)

    # # Create column with image name
    Image_Name = ImagePath.split('\\')[-1]
    Image_Name = [Image_Name] * num_contours

    # Geometric properties
    Labels = [rp.label for rp in props_contours]
    Areas = [rp.area for rp in props_contours]
    MajorAxes = [rp.major_axis_length for rp in props_contours]
    MinorAxes = [rp.minor_axis_length for rp in props_contours]
    Orientations = [rp.orientation for rp in props_contours]
    Perimeters = [rp.perimeter for rp in props_contours]
    Eccentricities = [rp.eccentricity for rp in props_contours]

    # Spectral properties
    red_props = regionprops(labeled_contours, intensity_image=cropped_rgb[:,:,0])
    green_props = regionprops(labeled_contours, intensity_image=cropped_rgb[:,:,1])
    blue_props = regionprops(labeled_contours, intensity_image=cropped_rgb[:,:,2])
    L_props = regionprops(labeled_contours, intensity_image=cropped_lab[:,:,0])
    a_props = regionprops(labeled_contours, intensity_image=cropped_lab[:,:,1])
    b_props = regionprops(labeled_contours, intensity_image=cropped_lab[:,:,2])
    H_props = regionprops(labeled_spks, intensity_image=cropped_hsv[:,:,0])
    S_props = regionprops(labeled_spks, intensity_image=cropped_hsv[:,:,1])
    V_props = regionprops(labeled_spks, intensity_image=cropped_hsv[:,:,2])
    
    red = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in red_props])
    green = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in green_props])
    blue = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in blue_props])
    L = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in L_props])
    a = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in a_props])
    b = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in b_props])
    H = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in H_props])
    S = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in S_props])
    V = np.array([[rp.mean_intensity,rp.min_intensity,rp.max_intensity] for rp in V_props])
    
    Red_Perc = np.array(channel_percentiles(red_props, Negatives=False)).T
    Green_Perc = np.array(channel_percentiles(green_props, Negatives=False)).T
    Blue_Perc = np.array(channel_percentiles(blue_props, Negatives=False)).T
    L_Perc = np.array(channel_percentiles(L_props)).T
    a_Perc = np.array(channel_percentiles(a_props, Negatives=True)).T
    b_Perc = np.array(channel_percentiles(b_props, Negatives=True)).T
    H_Perc = np.array(channel_percentiles(H_props)).T
    S_Perc = np.array(channel_percentiles(S_props)).T
    V_Perc = np.array(channel_percentiles(V_props)).T

    # Dataframe 1: for single obervation per spike
    Spikes_per_image = pd.DataFrame(
    list(zip(Image_Name, Labels, Areas, MajorAxes, MinorAxes, Orientations, Eccentricities, Perimeters, 
             red[:,0], red[:,1], red[:,2], green[:,0], green[:,1], green[:,2], blue[:,0], blue[:,1], blue[:,2], 
             L[:,0], L[:,1], L[:,2], a[:,0], a[:,1], a[:,2], b[:,0], b[:,1], b[:,2], 
             H[:,0], H[:,1], H[:,2], S[:,0], S[:,1], S[:,2], V[:,0], V[:,1], V[:,2], 
             Red_Perc[:,0], Red_Perc[:,1], Red_Perc[:,2], Red_Perc[:,3], Red_Perc[:,4], Red_Perc[:,5], Red_Perc[:,6], 
             Green_Perc[:,0], Green_Perc[:,1], Green_Perc[:,2], Green_Perc[:,3], Green_Perc[:,4], Green_Perc[:,5], Green_Perc[:,6], 
             Blue_Perc[:,0], Blue_Perc[:,1], Blue_Perc[:,2], Blue_Perc[:,3], Blue_Perc[:,4], Blue_Perc[:,5], Blue_Perc[:,6], 
             L_Perc[:,0], L_Perc[:,1], L_Perc[:,2], L_Perc[:,3], L_Perc[:,4], L_Perc[:,5], L_Perc[:,6], 
             a_Perc[:,0], a_Perc[:,1], a_Perc[:,2], a_Perc[:,3], a_Perc[:,4], a_Perc[:,5], a_Perc[:,6], 
             a_Perc[:,7], a_Perc[:,8], a_Perc[:,9], a_Perc[:,10], a_Perc[:,11], a_Perc[:,12], a_Perc[:,13], 
             b_Perc[:,0], b_Perc[:,1], b_Perc[:,2], b_Perc[:,3], b_Perc[:,4], b_Perc[:,5], b_Perc[:,6], 
             b_Perc[:,7], b_Perc[:,8], b_Perc[:,9], b_Perc[:,10], b_Perc[:,11], b_Perc[:,12], b_Perc[:,13],
             H_Perc[:,0], H_Perc[:,1], H_Perc[:,2], H_Perc[:,3], H_Perc[:,4], H_Perc[:,5], H_Perc[:,6],
             S_Perc[:,0], S_Perc[:,1], S_Perc[:,2], S_Perc[:,3], S_Perc[:,4], S_Perc[:,5], S_Perc[:,6],
             V_Perc[:,0], V_Perc[:,1], V_Perc[:,2], V_Perc[:,3], V_Perc[:,4], V_Perc[:,5], V_Perc[:,6])), 
    columns = ['Image_Name', 'Spike_Label', 'Area', 'MajorAxis', 'MinorAxes', 'Orientation', 'Eccentricity', 'Perimeter', 
               'Red_mean', 'Red_min', 'Red_max', 'Green_mean', 'Green_min', 'Green_max', 'Blue_mean', 'Blue_min', 'Blue_max', 
               'L_mean', 'L_min', 'L_max', 'a_mean', 'a_min', 'a_max', 'b_mean', 'b_min', 'b_max',
               'H_mean', 'H_min', 'H_max', 'S_mean', 'S_min', 'S_max', 'V_mean', 'V_min', 'V_max', 
               'Red_p25', 'Red_p50', 'Red_p75', 'Red_Mean', 'Red_sd', 'Red_Min', 'Red_Max', 
               'Green_p25', 'Green_p50', 'Green_p75', 'Green_Mean', 'Green_sd', 'Green_Min', 'Green_Max', 
               'Blue_p25', 'Blue_p50', 'Blue_p75', 'Blue_Mean', 'Blue_sd', 'Blue_Min', 'Blue_Max', 
               'L_p25', 'L_p50', 'L_p75', 'L_Mean', 'L_sd', 'L_Min', 'L_Max', 
               'a_p25_pos', 'a_p50_pos', 'a_p75_pos', 'a_Mean_pos', 'a_sd_pos', 'a_Min_pos', 'a_Max_pos', 
               'a_p25_neg', 'a_p50_neg', 'a_p75_neg', 'a_Mean_neg', 'a_sd_neg', 'a_Min_neg', 'a_Max_neg', 
               'b_p25_pos', 'b_p50_pos', 'b_p75_pos', 'b_Mean_pos', 'b_sd_pos', 'b_Min_pos', 'b_Max_pos', 
               'b_p25_neg', 'b_p50_neg', 'b_p75_neg', 'b_Mean_neg', 'b_sd_neg', 'b_Min_neg', 'b_Max_neg',
               'H_p25', 'H_p50', 'H_p75', 'H_Mean', 'H_sd', 'H_Min', 'H_Max',
               'S_p25', 'S_p50', 'S_p75', 'S_Mean', 'S_sd', 'S_Min', 'S_Max',
               'V_p25', 'V_p50', 'V_p75', 'V_Mean', 'V_sd', 'V_Min', 'V_Max'])

    Objects_per_image['Circularity'] = (4 * np.pi * Objects_per_image['Area']) / (Objects_per_image['Perimeter'] ** 2)

    
    # Unique labels
    labels2 = np.unique(labeled[labeled > 0])

    # Empty list for contours
    C = []

    # Loop thorugh labels and add to list of contours
    for label2 in labels2:
            y = labeled==label2
            y = y * 255
            y = y.astype('uint8')
            contours, hierarchy = cv2.findContours(y, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # len(contours)
            contours = np.squeeze(contours)
            C.append(contours)
        
    # List for angles
    Slopes = []
    
    contours = C    

    for c in contours:

        ellipse = cv2.fitEllipse(c)

        # Fit a line 
        rows,cols = cropped_rgb.shape[:2]
        [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01);
        lefty = int((-x*vy/vx) + y)
        righty = int(((cols-x)*vy/vx)+y)

        rise = (0,lefty)[1] - (cols-1,righty)[1]
        run = cols
        Slope = rise/run
        Slopes.append(Slope)
    
    # Add slopes to data frame
    Objects_per_image['Spklt_Angle'] = Slopes
    
    
    
    
    
    
    # Filter by area
    Objects_per_image2 = Objects_per_image[Objects_per_image['Area'] > MinSize]
    
    # Remove first row, corresponding to spikes' envelope
    if rm_envelope==True:
        return Objects_per_image2.iloc[1: , :]
    else:
        return Objects_per_image2


# plt.imshow(spklts)
# Example
# Props = ObjProps(spklts, cropped_rgb, cropped_lab, ImagePath=img_name, MinSize = 5000)
# Props = ObjProps(labeled=thresh2, cropped_rgb=cropped_rgb, cropped_lab=cropped_lab, ImagePath=img_name, MinSize = 1000)

## `EnhanceImage`

In [ ]:
%%time
def EnhanceImage(InputImage, Color = None, Contrast = None, Sharp = None):
    
    # Read image
    if isinstance(InputImage, str) == True:
        img = iio.imread(InputImage)
    else: 
        img = InputImage
    
    # RGB enhancement
    img0 = Image.fromarray(img)
    
    # Color seems to be good around 3.5
    img1 = ImageEnhance.Color(img0)
    if Color is not None:
        img1 = img1.enhance(Color)
    else:
        img1 = img0
    
    # Contrast
    img2 = ImageEnhance.Contrast(img1)
    if Contrast is not None:
        img2 = img2.enhance(Contrast)
    else:
        img2 = img1
    
    # Sharpness (Good ~20 or higher)
    img3 = ImageEnhance.Sharpness(img2)    
    if Sharp is not None:
        img3 = img3.enhance(Sharp)
    else:
        img3 = img2
    
    # Final image
    img3 = np.array(img3)
    
    return img3

# Example
# enh0 = EnhanceImage(InputImage=cropped_rgb, Color = 3, Contrast = None, Sharp = 10)
# plt.imshow(enh0)

## `LabelSpklts`
This fuction uses the Watershed algorithm to detect spikelets

In [ ]:
%%time

def LabelSpklts(cropped_rgb, MinDist=50, labels_out=True, n_spklt=True, ElliPlot=False, Plot=True):
    
    # Rescale to 10% of original
    rescaled_spk = rescale(cropped_rgb[...], 0.1, preserve_range=False, multichannel=True, anti_aliasing=True)
    # plt.imshow(rescaled_spk)

    # Erosion
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    erosion = cv2.erode(rescaled_spk,kernel,iterations = 1)
    # plt.imshow(erosion)

    # Opening
    kernel = np.ones((1,1),np.uint8)
    opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel, iterations = 10)
    # plt.imshow(opening)

    # Resize
    rescaled_spk2 = Image.fromarray((rescaled_spk * 255).astype(np.uint8))
    rescaled_spk2 = rescaled_spk2.resize((cropped_rgb.shape[1],cropped_rgb.shape[0]))
    # plt.imshow(rescaled_spk2)
    # rescaled_spk2.size
    opening = np.asarray(rescaled_spk2)
    # plt.imshow(opening)

    # Convert rgb to gray
    gray_spklts = opening @ [0.2126, 0.7152, 0.0722]
    # plt.imshow(gray_spklts)

    # Binarize gray
    bw_spklts = gray_spklts > 0
    # plt.imshow(bw_spklts)

    # Get distances
    distance = ndi.distance_transform_edt(bw_spklts)
    # plt.imshow(-distance)

    # Get max peaks
    coords = peak_local_max(distance, min_distance=MinDist, labels=bw_spklts)
    # plt.imshow(coords)

    mask = np.zeros(distance.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, spikelets = ndi.label(mask)
    
    # Watershed
    labels = watershed(-distance, markers, mask=cropped_spk)
#     plt.imshow(labels)

    labels2 = np.unique(labels[labels > 0])

    C = []

    for label in labels2:
            y = label_image == label
            y = y * 255
            y = y.astype('uint8')
            contours, hierarchy = cv2.findContours(y, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # len(contours)
            contours = np.squeeze(contours)
            C.append(contours)

#         plt.imshow(d[2])
    
    contours = C

    if ElliPlot==True and Plot==False:
        
        OutImage = cropped_rgb.copy()

        # Plot all found contours
        OutImage = cv2.drawContours(OutImage, contours, -1, (0,0,0), 10);
        # plt.imshow(OutImage)

        for c in contours:
            # Generate random colors
            random_channels = (np.random.choice(range(256), size=3))
            rr = int(random_channels[0])
            rg = int(random_channels[1])
            rb = int(random_channels[2])
            
            ellipse = cv2.fitEllipse(c)
            OutImage = cv2.ellipse(OutImage,ellipse,(rr,rg,rb),10);

            # Fit a line 
            rows,cols = OutImage.shape[:2]
            [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01);
            lefty = int((-x*vy/vx) + y)
            righty = int(((cols-x)*vy/vx)+y)
            # OutImage = cv2.line(OutImage,(cols-1,righty),(0,lefty),(rr,rg,rb),3);
            
            # Slope from tope left, which is is the origin [0,0]
            rise = (0,lefty)[1] - (cols-1,righty)[1]
            run = cols
            Slope = rise/run
            Slopes.append(Slope)            
        
        # Plot
        plt.imshow(OutImage)

    # Add slopes to data frame
    # Props['Spklt_Angle'] = Slopes

    if Plot==True and ElliPlot==False:  
        # Plot
        plt.imshow(labels, cmap=plt.cm.nipy_spectral)
        
        # Print number of spikelets detected
        print('Detected spikelets = ', spikelets)
    
    # Return labels
    if labels_out==True and n_spklt==True:
        return labels, spikelets

# Example:
# spklts, n_spklts= LabelSpklts(cropped_rgb, MinDist=50, labels_out=True, n_spklt=True, ElliPlot=True, Plot=False)
# plt.imshow(spklts==2)



## `CountorProps`
Contour properties

In [ ]:
%%time
def CountorProps(cropped_rgb, thresh2, ImagePath, ResizeFactor=30, MinSize = 1000, thr2=0.8, plot=True, rm_envelope=False):
    
#     labels_cont = SpkContours(cropped_rgb, ResizeFactor=30, MinSize = 1000, thr2=0.8, plot=True)
    
    # Copy image
    OutImage = cropped_rgb.copy()
    # Get lab
    OutImageLab = color.rgb2lab(cropped_lab)
    
    if thresh2 is not None:
        thresh2 = thresh2
    else:
        # Threshold for contours
        thresh2 = SpkltThresh(cropped=OutImage, ResizeFactor=ResizeFactor, thr2=thr2, MinSize=MinSize)

    #     plt.imshow(thresh2)
    
    # Enumerate objects
#     EnumerateSpkCV(thresh2, OutImage, TextSize=3, TROE2020=False)
    
#     contours = measure.find_contours(blurred,(0.8*255))
    contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     len(contours)
    # np.array(contours).shape
    # Contour properties
    Props = ObjProps(labeled=thresh2, cropped_rgb=cropped_rgb, cropped_lab=cropped_lab, ImagePath=ImagePath, rm_envelope=False)
    
    # Detected spikelets
    print("Fitted contours: ", len(contours))
    
    # Create list for slopes
    Slopes = []
#     len(Slopes)

    if plot==True:

        # Plot all found contours
        OutImage = cv2.drawContours(OutImage, contours, -1, (255,255,255), 10);

        for c in contours:
            # Generate random colors
            random_channels = (np.random.choice(range(256), size=3))
            rr = int(random_channels[0])
            rg = int(random_channels[1])
            rb = int(random_channels[2])
            
            ellipse = cv2.fitEllipse(c)
            OutImage = cv2.ellipse(OutImage,ellipse,(rr,rg,rb),10);

            # Fit a line 
            rows,cols = OutImage.shape[:2]
            [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01);
            lefty = int((-x*vy/vx) + y)
            righty = int(((cols-x)*vy/vx)+y)
            OutImage = cv2.line(OutImage,(cols-1,righty),(0,lefty),(rr,rg,rb),3);
            
            # Slope from tope left, which is is the origin [0,0]
            rise = (0,lefty)[1] - (cols-1,righty)[1]
            run = cols
            Slope = rise/run
            Slopes.append(Slope)            
        
        # Plot
        plt.imshow(OutImage)
        
    else:
        
        for c in contours:
            
            ellipse = cv2.fitEllipse(c)
            
            # Fit a line 
            rows,cols = OutImage.shape[:2]
            [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01);
            lefty = int((-x*vy/vx) + y)
            righty = int(((cols-x)*vy/vx)+y)

            rise = (0,lefty)[1] - (cols-1,righty)[1]
            run = cols
            Slope = rise/run
            Slopes.append(Slope)
    
    # Add slopes to data frame
    Props['Spklt_Angle'] = Slopes
    
    # Remove first row, corresponding to spikes' envelope
    if rm_envelope==True:
        return Props.iloc[1: , :]
    else:
        return Props



# Example:
# labels_cont = CountorProps(cropped_rgb, thresh2, ImagePath=img_name, ResizeFactor=30, MinSize = 1000, thr2=0.8, plot=True)

## `Heatmat`
Heatmaps from a 1d matrix. This is just for fun, at least for now. The idea is to create a matrix $A_{mxm}$ from a vector (or 1d matrix) $a_{m x 1}$, to plot $A$ under changing circumnstances, such as multiplying by increasing scalar, exponentiation, etc. This results in multiple frames that can then used to make a GIF.

In [ ]:
%%time

def Heatmat(a, frames=30):
    a = np.array(a)
    a = a.reshape(len(a), 1)
    aT = a.T
    mat = np.multiply(a, aT)
#     mata.shape

    for frame in range(frames):
        if frame < 10:
            name = "./GIFS/img_0"+str(frame)+".png"
        elif frame < 100:
            name = "./GIFS/img_00"+str(frame)+".png"
        else:
            name = "./GIFS/img_"+str(frame)+".png"
        
        new_mat = np.log10( 1+(mat**(frame) ) )
        sns.heatmap(new_mat)
        plt.savefig(name)
        plt.close()

# Example:
# Heatmat(a=Areas, frames=30)

## `makeGIF`
Create a GIF from images generated by `heatmat`

In [ ]:
%%time

def makeGIF(filenames, duration = 0.25, out_name=None):
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave('./GIFS/GIF.gif', images, duration=duration)


# Example:
# filenames = glob.glob("./GIFS/" + '*.png', recursive=False)
# makeGIF(filenames, duration = 0.25, out_name=None)

## `DistAll`
Distances among objects. This calculates the Euclidean distances (using the Pythagorean theorem) between all labeled ROIs given a binary image. If `spike_length` is given, the distances are returned as the proportion from the spike (%).

In [ ]:
%%time

def DistAll(bw, HeatMap=True, spike_length=None):
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(np.uint8(bw), 
                                                                               connectivity=8)
    # len(centroids[1:][:])
    img_center = centroids[0][:]
    c_points = centroids[1:][:]
    c_df = pd.DataFrame(c_points, columns=["x","y"])
    # c_df

    # https://stackoverflow.com/questions/57107729/how-to-compute-multiple-euclidean-distances-of-all-points-in-a-dataset

    # Consider points as tuples in a list
    data = [ (float(x),float(y)) for x,y in c_df[['x', 'y']].values ]

    # Create empty list for distances
    distances = []

    for point in data:

        # Compute the Euclidean distance between the current point and all others
        euc_dist = [math.sqrt((point[0]-x[0] )**2 + (point[1]-x[1])**2) for x in data]

        # Append to list
        distances.append(euc_dist)

    # Convert list to array
    D = np.array(distances)
    
    if spike_length != None:
        # Express it as a fraction from spike length
        D = D/spike_length
    else:
        D = D
    
    # Heatmap
    if HeatMap==True:
        sns.heatmap(D)
    
    return D

# Example:
# D = DistAll(bw=thresh2, HeatMap=True, spike_length=SL)
# D = DistAll(bw=spklts, HeatMap=True, spike_length=SL)

## `imgraph` Graph analysis (under contruction...)
Axes: `0` for $x$, `1` for $y$

In [ ]:
# https://stackoverflow.com/questions/54832694/how-to-represent-a-binary-image-as-a-graph-with-the-axis-being-height-and-width

def imgraph(image, axis=0):
    # image = blurred
    # Theshold image if desired (only 2d images)
    maxindex = np.argmax(image[:,:], axis=axis)

    # Plot graph
    plt.plot(image.shape[axis] - maxindex)
    plt.show()

## `ComparePlots`
Compare multiple plots. Plots need to be gathered in a list.

In [ ]:
%%time

def ComparePlots(rows, cols, ListImages, fontsize=10):
    plots = rows * cols
    fig, axes = plt.subplots(rows, cols, sharex=True, sharey=True)
    ax = axes.ravel()
    for i in range(plots):
        ax[i].imshow(ListImages[i], cmap='gray')
        Title = "Image " + str(i)
        ax[i].set_title(Title, fontsize=fontsize)
    fig.tight_layout()
    plt.show()
    
# Example:
# ComparePlots(3,1,[cropped_rgb, cropped_lab, cropped_hsv])

## `SeparateSpikes`
Split spikes into images. This function returns as many individual images as labeled in the mask.

In [ ]:
%%time

def SeparateSpikes(ImagePath, Outfile = None):
    
    # Remove bakground
    I = RemoveBackground(ImagePath, OtsuScaling=0.25, rgb_out=True, gray_out=True, lab_out=False, hsv_out=False, bw_out=True)
    rgb0 = I[0]
#     # Convert to gray
#     gray0 = img0 @ [0.2126, 0.7152, 0.0722]
# #     gray0 = img0 @ [0.2126, 0.7152, 0.0722]

#     # Threshold
#     otsu = filters.threshold_otsu(gray0)
#     bw0 = gray0 > otsu
#     bw1 = morphology.remove_small_objects(bw0, min_size=1.5e-05 * gray0.shape[0] * gray0.shape[1])
    bw0 = I[1]
#     plt.imshow(bw1)
    # Label spikes
    labeled_spks, num_spikes = label(bw0>0, return_num = True)
    
    # Loop through spikes
    for spk in range(1,num_spikes):
        
        # Select current spike
        myspk = labeled_spks == spk

        # Crop spike
        slice_x, slice_y = ndimage.find_objects(myspk)[0]
        cropped_spk = myspk[slice_x, slice_y]
        cropped_rgb = rgb0[slice_x, slice_y]
        cropped_rgb = np.where(cropped_spk[..., None], cropped_rgb, 0)
        
        # Add 10 pixels to each border
        padded = np.pad(cropped_rgb, pad_width=[(10, 10),(10, 10),(0, 0)], mode='constant')
        
        # Save image 
        im = Image.fromarray(padded)
        
        if Outfile == None:
            
            Split_Path = ImagePath.split("\\")
            OutName = Split_Path[-1].replace(".tif", "")
            Split_Path = Split_Path[:-1]
            OutDir = '\\'.join([str(i) for i in Split_Path])
            OutDir = OutDir + "\\IndividualSpikes\\"
            path = pathlib.Path(OutDir)
            path.mkdir(parents=True, exist_ok=True)
            
            if spk < 10:
                OutName = OutDir + OutName + "_spk0" + str(spk) + '.jpg'
            else: 
                OutName = OutDir + OutName + "_spk" + str(spk) + '.jpg'
        
        im.save(OutName)
        print("Saved image as: " + OutName)

# Example:
# SeparateSpikes(ImagePath=img_name)

# BATCH EXECUTION

In [ ]:
%%time

from SpykFunctions import *

path = r'./Images/TEST'
Images = ListImages(path, imgformat=".tif", recursive=False)
TROE2020=False

Spikes_data = pd.DataFrame()
Contours_data = pd.DataFrame()
Spklts_data = pd.DataFrame()
Distances_data = pd.DataFrame()
Lengths_data = pd.DataFrame()

for img_name in Images:
    # img_name=Images[0]
    print("Processing image ", img_name)
    
    # Remove background and create images
    I = RemoveBackground(img_name, OtsuScaling=0.25, rgb_out=True, gray_out=True, lab_out=True, hsv_out=True, bw_out=True)
    rgb0 = I[0]
    gray0 = I[1]
    lab0 = I[2]
    hsv0 = I[3]
    bw0 = I[4]
    
    # Enumerate spikes
#     EnumerateSpkCV(bw0, rgb0, TextSize=5, TROE2020=False)
    
    # Collect spikes data
    df = SpikesDF(I=I, ImagePath=img_name, RemoveBG=False)
    # Append to data set       
    Spikes_data = Spikes_data.append(df)

    # Label spikes
    labeled_spks, num_spikes = label(bw0, return_num = True)
    # plt.imshow(labeled_spks==2)
                  
    # Spike lengths
    SpkLengths = []
    
    # Spike distances from each other
    SpkDists = []

    # TROE2021 and PGR start at 2. Ignore background (0) and envelope (1)
    if TROE2020==False:
        FirstSpk = 1
    else:
        FirstSpk = 2
        
    # Start at 2 so it ignores background (0) and envelope (1)
    for Label in range(FirstSpk, num_spikes+1):
    
        print("Processing spike ", Label)
#         plt.imshow(labeled_spks==2)
        
        spk = labeled_spks==Label
        # plt.imshow(spk)
    
        # Crop spike
        slice_x, slice_y = ndimage.find_objects(spk)[0]
        cropped_spk = spk[slice_x, slice_y]
        cropped_rgb = rgb0[slice_x, slice_y]
        cropped_rgb = np.where(cropped_spk[..., None], cropped_rgb, 0)
        cropped_gray = color.rgb2gray(cropped_rgb)
        cropped_lab = color.rgb2lab(cropped_rgb)
        cropped_hsv = color.rgb2hsv(cropped_rgb)
#         plt.imshow(cropped_spk)

        # Spike length
        sl = spk_length(cropped_spk, method='skelblur', Overlay=False, PlotCH=False)
        SpkLengths.append(sl)

        # Theshold for detection
        thresh2 = SpkltThresh(cropped_rgb, ResizeFactor=30, thr2=0.8, MinSize=1000)

        # Distances between contours
        D = DistAll(bw=thresh2, HeatMap=False, spike_length=sl)
        SpkDists.append(D)

        # Contours
        labels_cont = CountorProps(cropped_rgb, cropped_lab, cropped_hsv, thresh2, ImagePath=img_name, ResizeFactor=30, MinSize = 1000, thr2=0.8, plot=False)

        # Watershed segments
        spklts, n_spklts= LabelSpklts(cropped_rgb, MinDist=50, labels_out=True, n_spklt=True, Plot=False)
        WSProps = ObjProps(labeled=spklts, cropped_rgb=cropped_rgb, cropped_lab=cropped_lab, cropped_hsv=cropped_hsv, ImagePath=img_name, MinSize = 5000)
        
    Contours_data = Contours_data.append(labels_cont)
    Spklts_data = Spklts_data.append(WSProps)
    Distances_data = Distances_data.append(SpkDists)
    Lengths_data = Lengths_data.append(SpkLengths)


# For one spike
# labeled_contours, num_contours = label(cropped_spk, return_num = True)
# red_props = regionprops(labeled_contours, intensity_image=cropped_rgb)
# Red_Perc = np.array(channel_percentiles(channel_props=red_props, Negatives=False)).T
# len(Red_Perc)
    

Processing image  ./Images/TEST\S2p13T4R3.tif
Processing spike  1


error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\shapedescr.cpp:360: error: (-201:Incorrect size of input array) There should be at least 5 points to fit the ellipse in function 'cv::fitEllipseNoDirect'


# DEMO

## Files
List images and select an image (`img_number`)

In [ ]:
%%time

# List images
path = r'./Images/TEST'
Images = ListImages(path, imgformat=".tif", recursive=False)
len(Images)

# Select image
img_number = 4
img_name = Images[img_number]

## Generate mask and color arrays
Remove the background and create rgb, gray, lab, hsv, and bw images.

In [ ]:
%%time
I = RemoveBackground(img_name, OtsuScaling=0.25, rgb_out=True, gray_out=True, lab_out=True, hsv_out=True, bw_out=True)
rgb0 = I[0]
gray0 = I[1]
lab0 = I[2]
hsv0 = I[3]
bw0 = I[4]


## Enumerate spikes

In [ ]:
%%time
# Enumerate spikes
EnumerateSpkCV(bw0, rgb0, TextSize=5, TROE2020=False)

In [ ]:
%%time
PixelHist(bw=bw0, Lab=lab0, channel = 0, spikes=[1,2,26], nbins = 100)

## Get spike properties

In [ ]:
%%time
df = SpikesDF(I, ImagePath, RemoveBG=False, PrintSpkLabels=False)

In [ ]:
%%time
df
# notice that the envelope (Spike_Label 1) was removed.

## Select a spike

In [ ]:
%%time
# Select a spike from the labeled image
Selected_Spike = 26
print("Selected spike number", Selected_Spike)

# Label spikes
labeled_spks, num_spikes = label(bw0, return_num = True)
spk = labeled_spks==Selected_Spike

# Crop spike
slice_x, slice_y = ndimage.find_objects(spk)[0]
cropped_spk = spk[slice_x, slice_y]
cropped_rgb = rgb0[slice_x, slice_y]
cropped_rgb = np.where(cropped_spk[..., None], cropped_rgb, 0)
cropped_gray = cropped_rgb @ [0.2126, 0.7152, 0.0722]
cropped_lab = color.rgb2lab(cropped_rgb)
cropped_hsv = color.rgb2hsv(cropped_rgb)

# Visualize selected spike
plt.imshow(cropped_rgb)

## Spike length

In [ ]:
%%time
L = spk_length(cropped_spk, method='skelblur', Overlay=True, PlotCH=False)

## Threshold for contour detection

In [ ]:
%%time
thresh2 = SpkltThresh(cropped=cropped_rgb, ResizeFactor=30, thr2=0.8, MinSize=1000)
plt.imshow(thresh2)

### Contour for spikelet detection

In [ ]:
%%time
labels_cont = LabelContours(cropped_rgb, ResizeFactor=30, MinSize = 1000, thr2=0.8, plot=True)

### Get contour properties

In [ ]:
%time
CountProp = SpkltProps(cropped_rgb=cropped_rgb, ImagePath=img_name, ResizeFactor=30, MinSize = 1000, plot=True)

In [ ]:
CountProp

### Distance between detected spikelets

In [ ]:
# Example:
D = DistAll(bw=thresh2, HeatMap=True, spike_length=SL)

In [ ]:
EnumerateSpkCV(thresh2, cropped_rgb, TextSize=3,  TROE2020=False)

## Watershed for spikelet detection

In [ ]:
%%time
# Using watershed
spklts, n_spklts= LabelSpklts(cropped_rgb, MinDist=50, labels_out=True, n_spklt=True, Plot=True)

### Spikelet properties

In [ ]:
%%time
WSProps = ObjProps(labeled=spklts, cropped_rgb=cropped_rgb, cropped_lab=cropped_lab, ImagePath=img_name, MinSize = 10000)

In [ ]:
WSProps
# Notice that the number of rows is lower. That's because the elements with that don't meet MinSize are removed.